In [ ]:
# =======================
# S0. Imports & paths
# =======================

import numpy as np
import pandas as pd
from pathlib import Path
from dataclasses import dataclass

# Root folder for outputs (figures, CSVs, etc.)
OUT_ROOT = Path("./results/notebook_run")
OUT_ROOT.mkdir(parents=True, exist_ok=True)

# Root folder where your telemetry CSVs live
DATA_ROOT = Path("./data/telemetry")  # <-- set this to your telemetry folder

print("[S0] DATA_ROOT =", DATA_ROOT)
print("[S0] Available CSV files:")
csv_files = sorted(DATA_ROOT.glob("*.csv"))
for p in csv_files:
    print("   -", p.name)

RANDOM_SEED = 123
np.random.seed(RANDOM_SEED)

# Meta columns (used later in S2+)
META_COLS_BASE = {
    "setup",
    "scenario",
    "workload",
    "time_idx",
    "label",
    "is_anom",
}


# =======================
# S1. Load telemetry CSVs
# =======================

def _parse_scenario_workload_from_name(path: Path):
    """
    Expect filenames like:
        DDR4_DROOP_dft.csv
        DDR4_benign_tr.csv
        DDR5_SPECTRE_mm.csv
        DDR5_benign_ni.csv
    Returns (scenario_str, workload_str).
    """
    stem = path.stem  # e.g., 'DDR4_DROOP_dft'
    parts = stem.split("_")
    if len(parts) < 3:
        raise ValueError(f"Cannot parse scenario/workload from filename: {path.name}")

    # parts[0] = 'DDR4' or 'DDR5'
    scen_raw = parts[1]      # e.g., 'DROOP', 'RH', 'SPECTRE', 'benign'
    wl_raw = parts[2]        # e.g., 'dft', 'dj', ...

    scenario = scen_raw.strip().upper()
    if scenario == "BENIGN":
        scenario = "BENIGN"  # normalize (already upper anyway)
    # DROOP / RH / SPECTRE are already upper in your names

    workload = wl_raw.strip().upper()  # DFT, DJ, ...

    return scenario, workload


def load_telemetry_for_setup(setup: str, data_root: Path) -> pd.DataFrame:
    """
    Build a single telemetry DataFrame for a given setup:
      - setup='A' -> all DDR4_* CSVs
      - setup='B' -> all DDR5_* CSVs

    Adds columns:
      - setup        ('A' or 'B')
      - scenario     ('BENIGN', 'DROOP', 'RH', 'SPECTRE')
      - workload     ('DFT', 'DJ', ...)
      - time_idx     (0..N-1 within each file)
      - is_anom      (0 for BENIGN, 1 for others)
      - label        (same as is_anom at sample level)
    """
    if setup.upper() == "A":
        prefix = "DDR4_"
    elif setup.upper() == "B":
        prefix = "DDR5_"
    else:
        raise ValueError(f"Unknown setup: {setup}. Expected 'A' or 'B'.")

    files = sorted(data_root.glob(f"{prefix}*.csv"))
    if not files:
        raise FileNotFoundError(
            f"[S1] No files found for setup {setup} with prefix {prefix} in {data_root}"
        )

    print(f"[S1] Loading setup {setup} from {len(files)} files with prefix {prefix}*")

    dfs = []
    for path in files:
        scenario, workload = _parse_scenario_workload_from_name(path)
        df = pd.read_csv(path)

        # Add meta columns
        df["setup"] = setup
        df["scenario"] = scenario
        df["workload"] = workload

        # time_idx: if missing, just use row order within that file
        if "time_idx" not in df.columns:
            df["time_idx"] = np.arange(len(df), dtype=int)

        # Labels: BENIGN = 0, all other scenarios = 1
        is_anom = 0 if scenario == "BENIGN" else 1
        df["is_anom"] = is_anom
        df["label"] = is_anom

        dfs.append(df)

        print(
            f"[S1]   {path.name}: scenario={scenario}, workload={workload}, "
            f"rows={len(df)}"
        )

    full_df = pd.concat(dfs, ignore_index=True)

    # Sort for reproducible windowing later
    full_df = full_df.sort_values(
        ["workload", "scenario", "time_idx"]
    ).reset_index(drop=True)

    print(
        f"[S1] Final telemetry for setup {setup}: "
        f"{len(full_df)} rows, {full_df.shape[1]} columns."
    )
    print(
        f"[S1]   Scenarios: {sorted(full_df['scenario'].unique())}, "
        f"Workloads: {sorted(full_df['workload'].unique())}"
    )

    return full_df


# ---- Load A and B from the folder ----
telemetry_A = load_telemetry_for_setup("A", DATA_ROOT)  # DDR4_* -> Setup A
telemetry_B = load_telemetry_for_setup("B", DATA_ROOT)  # DDR5_* -> Setup B


In [ ]:
# ============================
# S2. Cleaning & debiasing
# ============================

def get_feature_columns(df: pd.DataFrame) -> list[str]:
    """
    Numeric telemetry columns used as features, excluding meta/label columns.
    """
    meta_cols = set(META_COLS_BASE)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    feature_cols = [c for c in numeric_cols if c not in meta_cols]
    return feature_cols


def clean_and_debias_telemetry(df: pd.DataFrame) -> pd.DataFrame:
    """
    Simple cleaning + benign debiasing:
      - Drop rows with all-NaN numeric features
      - Subtract per-feature BENIGN mean (if available)
    Returns a new DataFrame.
    """
    df = df.copy()

    feature_cols = get_feature_columns(df)
    if not feature_cols:
        raise ValueError("No numeric feature columns found for telemetry.")

    # Drop rows where all feature columns are NaN
    mask_all_nan = df[feature_cols].isna().all(axis=1)
    if mask_all_nan.any():
        df = df.loc[~mask_all_nan].reset_index(drop=True)
        print(f"[S2] Dropped {mask_all_nan.sum()} rows with all-NaN features.")

    # Compute benign means (if any BENIGN rows exist)
    if "scenario" in df.columns:
        ben_mask = df["scenario"] == "BENIGN"
        if ben_mask.any():
            benign_means = df.loc[ben_mask, feature_cols].mean(axis=0)
            df[feature_cols] = df[feature_cols].fillna(benign_means)
            df[feature_cols] = df[feature_cols] - benign_means
            print(
                f"[S2] Debiased features using BENIGN mean for "
                f"{ben_mask.sum()} benign rows."
            )
        else:
            # Fallback: use global mean if no BENIGN rows
            global_means = df[feature_cols].mean(axis=0)
            df[feature_cols] = df[feature_cols].fillna(global_means)
            df[feature_cols] = df[feature_cols] - global_means
            print("[S2] No BENIGN rows; debiased using global feature means.")
    else:
        # Very unlikely given S1, but just in case
        global_means = df[feature_cols].mean(axis=0)
        df[feature_cols] = df[feature_cols].fillna(global_means)
        df[feature_cols] = df[feature_cols] - global_means
        print("[S2] 'scenario' missing; debiased using global feature means.")

    return df


telemetry_A_Z = clean_and_debias_telemetry(telemetry_A)
telemetry_B_Z = clean_and_debias_telemetry(telemetry_B)

feat_A = get_feature_columns(telemetry_A_Z)
feat_B = get_feature_columns(telemetry_B_Z)
shared_features = sorted(set(feat_A) & set(feat_B))

print(f"[S2] Setup A feature count: {len(feat_A)}")
print(f"[S2] Setup B feature count: {len(feat_B)}")
print(f"[S2] Shared features used by CIAS: {len(shared_features)}")


In [ ]:
# =======================
# S3. CIAS model
# =======================

@dataclass
class CIASModel:
    """
    Lightweight CIAS parameter bundle + scoring functions.

    feature_cols : list of feature names (same order as mu/sigma/w)
    mu           : benign mean per feature
    sigma        : benign std per feature
    w            : non-negative weights per feature (sum to 1)
    lambda_res   : trade-off between E2(t) and E1(t) in CIAS score
    """
    feature_cols: list
    mu: np.ndarray
    sigma: np.ndarray
    w: np.ndarray
    lambda_res: float = 0.5

    def _sigma_safe(self) -> np.ndarray:
        """Avoid divide-by-zero in normalization."""
        return np.where(self.sigma <= 1e-6, 1.0, self.sigma)

    def score_array(self, X: np.ndarray):
        """
        Compute (score, E1, E2) for a 2D array X of shape (n_samples, n_features).
        """
        s = self._sigma_safe()
        Z = (X - self.mu) / s

        # E2: weighted variance-normalized squared distance (Euclidean-type)
        E2 = np.sum(self.w * (Z ** 2), axis=1)

        # E1: weighted sum of absolute shifts (L1-type residual mass)
        E1 = np.sum(self.w * np.abs(Z), axis=1)

        score = (1.0 - self.lambda_res) * E2 + self.lambda_res * E1
        return score, E1, E2

    def score_dataframe(self, df: pd.DataFrame):
        """
        Convenience wrapper for DataFrames with matching feature columns.
        """
        X = df[self.feature_cols].to_numpy(dtype=float)
        return self.score_array(X)


In [ ]:
# ============================
# S4. Fit CIAS from BENIGN
# ============================

def fit_cias_from_benign(
    df: pd.DataFrame,
    feature_cols: list[str],
    lambda_res: float = 0.5,
) -> CIASModel:
    """
    Fit a CIASModel using only BENIGN rows:
      - mu, sigma from benign feature statistics
      - weights w_f from inverse benign variance (larger weight for
        more stable / less noisy features).
    """
    if "scenario" not in df.columns:
        raise KeyError("Expected 'scenario' column to fit CIAS.")

    ben = df[df["scenario"] == "BENIGN"].copy()
    if ben.empty:
        raise ValueError("No BENIGN rows found; cannot fit CIAS.")

    X_ben = ben[feature_cols].to_numpy(dtype=float)
    mu = X_ben.mean(axis=0)
    sigma = X_ben.std(axis=0, ddof=1)

    # Simple unsupervised weighting: inverse variance (more stable → more weight)
    inv_var = 1.0 / np.maximum(sigma ** 2, 1e-8)
    w_raw = np.maximum(inv_var, 0.0)
    if np.all(w_raw == 0):
        w = np.ones_like(w_raw) / len(w_raw)
    else:
        w = w_raw / w_raw.sum()

    model = CIASModel(
        feature_cols=list(feature_cols),
        mu=mu,
        sigma=sigma,
        w=w,
        lambda_res=float(lambda_res),
    )

    print(
        f"[S4] Fitted CIAS model with {len(feature_cols)} features, "
        f"lambda_res={lambda_res:.3f}"
    )
    return model


# Fit CIAS models for both setups using the same shared_features list
cias_A = fit_cias_from_benign(telemetry_A_Z, shared_features, lambda_res=0.5)
cias_B = fit_cias_from_benign(telemetry_B_Z, shared_features, lambda_res=0.5)


In [ ]:
# ===========================================
# S5. Attach per-sample CIAS scores (optional)
# ===========================================

def attach_cias_scores(
    df: pd.DataFrame,
    cias_model: CIASModel,
    score_col: str = "cias_sample_score",
) -> pd.DataFrame:
    """
    Add CIAS per-sample score, E1, and E2 columns to a telemetry DataFrame.
    """
    df = df.copy()
    scores, E1, E2 = cias_model.score_dataframe(df)
    df[score_col] = scores
    df[f"{score_col}_E1"] = E1
    df[f"{score_col}_E2"] = E2
    return df


telemetry_A_Z = attach_cias_scores(telemetry_A_Z, cias_A, score_col="cias_sample_score")
telemetry_B_Z = attach_cias_scores(telemetry_B_Z, cias_B, score_col="cias_sample_score")

print(
    f"[S5] telemetry_A_Z shape: {telemetry_A_Z.shape}, "
    f"columns added: ['cias_sample_score', 'cias_sample_score_E1', 'cias_sample_score_E2']"
)
print(
    f"[S5] telemetry_B_Z shape: {telemetry_B_Z.shape}, "
    f"columns added: ['cias_sample_score', 'cias_sample_score_E1', 'cias_sample_score_E2']"
)


In [ ]:
# ============================================
# S6–S_MASTER: EXACT CIAS windowing, DROOP tuning,
#              K-fold evaluation, summaries, driver
# ============================================

from dataclasses import dataclass
from pathlib import Path
from typing import Optional, List, Dict

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    f1_score,
    balanced_accuracy_score,
    matthews_corrcoef,
    brier_score_loss,
)

# ------------------------------------------------
# S6: CIAS per-sample scoring + window builder
# ------------------------------------------------

def _score_samples_with_cias(
    df: pd.DataFrame,
    cias_model,
    lambda_res: float = 0.5,
    score_col: str = "score_cias",
) -> np.ndarray:
    """
    Get per-sample CIAS-like scores for telemetry rows.

    Priority:
      1) If df has a precomputed per-sample CIAS column (score_col), use it.
      2) Otherwise, try the given CIAS model if it exposes a scoring API.
      3) If that fails, fall back to a fully self-contained scoring rule:

         - Use BENIGN rows to estimate mean / std per feature.
         - Compute z-scores for every sample.
         - Compute feature weights from BENIGN vs. non-BENIGN effect size.
         - Define:
             E1(t) = Σ_f w_f |z_f(t)|   (many small shifts)
             E2(t) = max_f w_f |z_f(t)| (single large spike)
           and
             score(t) = (1 - λ_res) E2(t) + λ_res E1(t).
    """
    # ---------------------------------------------------
    # 1) Use existing per-sample CIAS column if available
    # ---------------------------------------------------
    if score_col in df.columns:
        return df[score_col].to_numpy(dtype=float)

    # -----------------------------------------
    # 2) Try using the CIAS model if possible
    # -----------------------------------------
    # Build feature matrix if we can
    feat_cols = None
    X = None

    def _get_feat_matrix_from_df(df_local: pd.DataFrame) -> np.ndarray:
        nonlocal feat_cols
        meta_cols = {"setup", "scenario", "workload", "time_idx", "label"}
        feat_cols = [
            c
            for c in df_local.columns
            if c not in meta_cols and pd.api.types.is_numeric_dtype(df_local[c])
        ]
        if not feat_cols:
            raise ValueError(
                "No numeric feature columns found for CIAS scoring. "
                f"Columns={list(df_local.columns)}"
            )
        return df_local[feat_cols].to_numpy(dtype=float)

    if cias_model is not None:
        # If the model tells us which features to use, respect that
        if hasattr(cias_model, "feature_names_"):
            feat_cols = list(cias_model.feature_names_)
            X = df[feat_cols].to_numpy(dtype=float)
        else:
            X = _get_feat_matrix_from_df(df)

        # Try a series of method names
        try:
            if hasattr(cias_model, "score_samples"):
                try:
                    return np.asarray(
                        cias_model.score_samples(X, lambda_res=lambda_res),
                        dtype=float,
                    )
                except TypeError:
                    return np.asarray(cias_model.score_samples(X), dtype=float)

            if hasattr(cias_model, "score"):
                try:
                    return np.asarray(
                        cias_model.score(X, lambda_res=lambda_res),
                        dtype=float,
                    )
                except TypeError:
                    return np.asarray(cias_model.score(X), dtype=float)

            if hasattr(cias_model, "compute_scores"):
                return np.asarray(cias_model.compute_scores(X), dtype=float)

            if hasattr(cias_model, "predict_proba"):
                probs = np.asarray(cias_model.predict_proba(X), dtype=float)
                if probs.ndim == 2 and probs.shape[1] >= 2:
                    # Assume class 1 is anomaly
                    return probs[:, 1]
                return probs.reshape(-1)

            if hasattr(cias_model, "predict"):
                return np.asarray(cias_model.predict(X), dtype=float)
        except Exception:
            # If the model-based path fails for any reason, fall through
            pass

    # ---------------------------------------------------
    # 3) Fallback: self-contained CIAS-like scoring
    # ---------------------------------------------------
    # If we didn't build X above, do it now.
    if X is None:
        X = _get_feat_matrix_from_df(df)

    # We expect df to contain both BENIGN and anomaly scenario here
    scen_col = "scenario" if "scenario" in df.columns else None
    if scen_col is not None:
        scen_vals = df[scen_col].astype(str).str.upper()
        mask_ben = scen_vals == "BENIGN"
        mask_anom = ~mask_ben
    else:
        # If there is no scenario column, treat all as BENIGN for stats
        mask_ben = np.ones(len(df), dtype=bool)
        mask_anom = ~mask_ben

    # Baseline stats from BENIGN rows (or all rows if no benign subset)
    if mask_ben.any():
        X_ben = X[mask_ben]
    else:
        X_ben = X

    mu_b = X_ben.mean(axis=0)
    sigma_b = X_ben.std(axis=0)
    # Avoid division by zero
    sigma_b = np.where(sigma_b < 1e-6, 1.0, sigma_b)

    # Standardized residuals relative to benign
    Z = (X - mu_b) / sigma_b  # shape: (n_samples, n_features)

    # Feature weights from effect size between anomaly and benign
    if scen_col is not None and mask_anom.any():
        X_anom = X[mask_anom]
        mu_a = X_anom.mean(axis=0)
        w_raw = np.abs(mu_a - mu_b) / sigma_b  # effect size-like weight
    else:
        w_raw = np.ones_like(mu_b)

    w_raw = np.nan_to_num(w_raw, nan=0.0, posinf=0.0, neginf=0.0)

    if w_raw.sum() <= 0:
        w = np.ones_like(w_raw) / float(len(w_raw))
    else:
        w = w_raw / w_raw.sum()

    # Weighted absolute z-scores
    absZ = np.abs(Z) * w  # broadcasting over features

    # E1(t): many small shifts (weighted sum)
    E1 = absZ.sum(axis=1)

    # E2(t): single large spike (weighted max)
    E2 = absZ.max(axis=1)

    # CIAS-style mixture
    score = (1.0 - float(lambda_res)) * E2 + float(lambda_res) * E1

    return score.astype(float)



def build_window_dataset_for_setup(
    df_Z: pd.DataFrame,
    cias_model,
    scenario_anom: str,
    W: int,
    agg_mode: str = "mean",
    lambda_res: float = 0.5,
    balance_per_workload: bool = True,
    seed: int = 123,
) -> pd.DataFrame:
    """
    Build a benign vs anomaly window dataset for a given anomaly scenario.

    Assumes df_Z already contains one setup (setup column fixed).
    Uses non-overlapping windows per (workload, scenario).
    """
    rng = np.random.default_rng(seed)

    if "setup" not in df_Z.columns:
        raise KeyError("df_Z must contain a 'setup' column.")

    setup_vals = df_Z["setup"].unique()
    if len(setup_vals) != 1:
        raise ValueError(
            f"df_Z must contain exactly one setup; got {setup_vals}. "
            "Filter df_Z per setup before calling this function."
        )
    setup = setup_vals[0]

    # Keep only BENIGN and the chosen anomaly scenario
    scen_anom_u = scenario_anom.upper()
    scen_keep = ["BENIGN", scen_anom_u]
    df_sub = df_Z[df_Z["scenario"].isin(scen_keep)].copy()

    if df_sub.empty:
        print(
            f"[S6] No rows for setup={setup}, scenario_anom={scen_anom_u}; "
            "returning empty DataFrame."
        )
        return pd.DataFrame()

    if "workload" not in df_sub.columns:
        raise KeyError("df_Z must contain 'workload' column.")

    if "time_idx" not in df_sub.columns:
        # Safety: if not already present, create a local index
        df_sub["time_idx"] = df_sub.groupby(["scenario", "workload"]).cumcount()

    df_sub = df_sub.sort_values(
        ["workload", "scenario", "time_idx"]
    ).reset_index(drop=True)

    # Per-sample CIAS scores
    df_sub["score_sample"] = _score_samples_with_cias(
        df_sub, cias_model, lambda_res=lambda_res
    )

    rows = []

    for wl, df_w in df_sub.groupby("workload"):
        df_b = df_w[df_w["scenario"] == "BENIGN"]
        df_a = df_w[df_w["scenario"] == scen_anom_u]

        s_b = df_b["score_sample"].to_numpy(dtype=float)
        s_a = df_a["score_sample"].to_numpy(dtype=float)

        if s_b.size < W or s_a.size < W:
            # Not enough samples for this workload/scenario
            continue

        n_win_b = s_b.size // W
        n_win_a = s_a.size // W

        if balance_per_workload:
            n_win = min(n_win_b, n_win_a)
            n_win_b = n_win_a = n_win

        def _aggregate_window_scores(scores_1d: np.ndarray, n_win: int) -> np.ndarray:
            out = []
            for i in range(n_win):
                start = i * W
                end = start + W
                window_scores = scores_1d[start:end]
                if window_scores.size < W:
                    break
                if agg_mode == "mean":
                    s_win = float(window_scores.mean())
                elif agg_mode == "max":
                    s_win = float(window_scores.max())
                elif agg_mode == "median":
                    s_win = float(np.median(window_scores))
                else:
                    # Default back to mean
                    s_win = float(window_scores.mean())
                out.append(s_win)
            return np.array(out, dtype=float)

        win_scores_b = _aggregate_window_scores(s_b, n_win_b)
        win_scores_a = _aggregate_window_scores(s_a, n_win_a)

        for s in win_scores_b:
            rows.append(
                {
                    "setup": setup,
                    "scenario": scen_anom_u,
                    "workload": wl,
                    "window_size": int(W),
                    "agg_mode": agg_mode,
                    "lambda_res": float(lambda_res),
                    "score_win": float(s),
                    "label": 0,
                }
            )
        for s in win_scores_a:
            rows.append(
                {
                    "setup": setup,
                    "scenario": scen_anom_u,
                    "workload": wl,
                    "window_size": int(W),
                    "agg_mode": agg_mode,
                    "lambda_res": float(lambda_res),
                    "score_win": float(s),
                    "label": 1,
                }
            )

    win_df = pd.DataFrame(rows)
    print(
        f"[S6] Built window dataset for setup={setup}, scenario={scen_anom_u}, "
        f"W={W}: shape={win_df.shape}"
    )
    return win_df


# ------------------------------------------------
# S7: DROOP-only aggressive tuning
# ------------------------------------------------

@dataclass
class DroopConfig:
    setup: str
    window_size: int
    n_splits: int
    lambda_res: float
    agg_mode: str
    p_quantile: float
    auc_roc_mean: float
    auc_pr_mean: float
    f1_mean: float
    csv_path: Optional[Path] = None


def _compute_ece(y_true: np.ndarray, y_prob: np.ndarray, n_bins: int = 10) -> float:
    """
    Expected Calibration Error (ECE) for binary probs.
    y_true: {0,1}
    y_prob: predicted probabilities in [0,1]
    """
    y_true = np.asarray(y_true).astype(int)
    y_prob = np.asarray(y_prob).astype(float)

    if y_true.size == 0:
        return np.nan

    if np.unique(y_true).size < 2:
        # Degenerate: only one class
        return 0.0

    bins = np.linspace(0.0, 1.0, n_bins + 1)
    ece = 0.0
    n = y_true.size

    for i in range(n_bins):
        lo, hi = bins[i], bins[i + 1]
        if i < n_bins - 1:
            mask = (y_prob >= lo) & (y_prob < hi)
        else:
            mask = (y_prob >= lo) & (y_prob <= hi)

        if not np.any(mask):
            continue
        p_bin = y_prob[mask].mean()
        y_bin = y_true[mask].mean()
        w_bin = mask.mean()
        ece += w_bin * abs(p_bin - y_bin)

    return float(ece)


def _safe_binary_metric(fn, y_true, y_score_or_pred, default=np.nan, **kwargs):
    y_true = np.asarray(y_true).astype(int)
    if y_true.size == 0 or np.unique(y_true).size < 2:
        return default
    try:
        return float(fn(y_true, y_score_or_pred, **kwargs))
    except Exception:
        return default


def _compute_window_metrics(y_true, y_pred, y_prob):
    """
    Compute detection metrics given labels, predictions, and probabilities.
    """
    y_true = np.asarray(y_true).astype(int)
    y_pred = np.asarray(y_pred).astype(int)
    y_prob = np.asarray(y_prob).astype(float)

    auc_roc = _safe_binary_metric(roc_auc_score, y_true, y_prob, default=np.nan)
    auc_pr = _safe_binary_metric(average_precision_score, y_true, y_prob, default=np.nan)
    f1 = _safe_binary_metric(f1_score, y_true, y_pred, default=0.0)
    bal_acc = _safe_binary_metric(balanced_accuracy_score, y_true, y_pred, default=0.5)
    mcc = _safe_binary_metric(matthews_corrcoef, y_true, y_pred, default=0.0)
    brier = _safe_binary_metric(brier_score_loss, y_true, y_prob, default=np.nan)
    ece = _compute_ece(y_true, y_prob, n_bins=10)

    return {
        "auc_roc": auc_roc,
        "auc_pr": auc_pr,
        "f1": f1,
        "bal_acc": bal_acc,
        "mcc": mcc,
        "brier": brier,
        "ece": ece,
    }


def evaluate_windows_kfold(
    win_df: pd.DataFrame,
    setup: str,
    scenario_eval: str,
    W: int,
    agg_mode: str,
    lambda_res: float,
    p_quantile: float,
    n_splits: int,
    seed: int,
) -> List[Dict]:
    """
    Evaluate CIAS window scores with K-fold CV.

    Returns a list of dict rows containing BOTH:
      - workload='ALL' rows (global performance)
      - workload='<wl>' rows (per-workload performance)
    """
    if "score_win" not in win_df.columns:
        raise KeyError(
            "[S8] win_df must contain a 'score_win' column. "
            f"Columns={list(win_df.columns)}"
        )
    if "label" not in win_df.columns:
        raise KeyError(
            "[S8] win_df must contain a 'label' column. "
            f"Columns={list(win_df.columns)}"
        )

    scores = win_df["score_win"].to_numpy().astype(float)
    labels = win_df["label"].to_numpy().astype(int)

    if "workload" in win_df.columns:
        workloads = win_df["workload"].astype(str).to_numpy()
    else:
        workloads = np.array(["ALL"] * len(win_df))

    if np.unique(labels).size < 2:
        print(
            f"[S8] WARNING: only one class present for setup={setup}, "
            f"scenario={scenario_eval}, W={W}; skipping."
        )
        return []

    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    rows: List[Dict] = []

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(scores, labels)):
        train_scores = scores[train_idx]
        train_labels = labels[train_idx]

        ben_train = train_scores[train_labels == 0]
        if ben_train.size == 0:
            tau = np.quantile(train_scores, p_quantile)
        else:
            tau = np.quantile(ben_train, p_quantile)

        test_scores = scores[test_idx]
        test_labels = labels[test_idx]
        test_workloads = workloads[test_idx]

        # Min-max scaling on train scores to create pseudo-probabilities
        s_min, s_max = train_scores.min(), train_scores.max()
        denom = (s_max - s_min) if (s_max > s_min) else 1.0
        test_probs = (test_scores - s_min) / denom
        test_probs = np.clip(test_probs, 0.0, 1.0)

        test_pred = (test_scores >= tau).astype(int)

        # ---- Global row (workload='ALL') ----
        metrics_all = _compute_window_metrics(test_labels, test_pred, test_probs)
        row_all = {
            "setup": setup,
            "scenario": scenario_eval,
            "workload": "ALL",
            "window_size": int(W),
            "n_splits": int(n_splits),
            "fold_idx": int(fold_idx),
            "agg_mode": agg_mode,
            "lambda_res": float(lambda_res),
            "p_quantile": float(p_quantile),
            "n_test": int(test_labels.size),
            "n_benign": int((test_labels == 0).sum()),
            "n_anom": int((test_labels == 1).sum()),
        }
        row_all.update(metrics_all)
        rows.append(row_all)

        # ---- Per-workload rows ----
        for wl in np.unique(test_workloads):
            mask_w = test_workloads == wl
            y_w = test_labels[mask_w]
            s_w = test_scores[mask_w]
            p_w = test_probs[mask_w]
            pred_w = test_pred[mask_w]

            if y_w.size < 2 or np.unique(y_w).size < 2:
                continue

            metrics_w = _compute_window_metrics(y_w, pred_w, p_w)
            row_w = {
                "setup": setup,
                "scenario": scenario_eval,
                "workload": wl,
                "window_size": int(W),
                "n_splits": int(n_splits),
                "fold_idx": int(fold_idx),
                "agg_mode": agg_mode,
                "lambda_res": float(lambda_res),
                "p_quantile": float(p_quantile),
                "n_test": int(y_w.size),
                "n_benign": int((y_w == 0).sum()),
                "n_anom": int((y_w == 1).sum()),
            }
            row_w.update(metrics_w)
            rows.append(row_w)

    return rows


def tune_droop_for_setup(
    setup: str,
    df_Z: pd.DataFrame,
    cias_model,
    window_size: int = 500,
    n_splits: int = 5,
    lambda_grid: Optional[List[float]] = None,
    agg_mode_grid: Optional[List[str]] = None,
    p_grid: Optional[List[float]] = None,
    seed: int = 123,
) -> Optional[DroopConfig]:
    """
    DROOP-only aggressive tuning for a given setup, using a grid over:
      - λ_res
      - aggregation mode
      - benign-tail quantile p (threshold)
    """
    print(f"\n[S7] DROOP tuning for setup {setup} (W={window_size}, k={n_splits})...")

    if lambda_grid is None:
        lambda_grid = list(np.linspace(0.0, 1.0, 11))  # 0.0,0.1,...,1.0
    if agg_mode_grid is None:
        agg_mode_grid = ["mean", "max"]
    if p_grid is None:
        p_grid = [0.85, 0.90, 0.95, 0.97, 0.99]

    df_setup = df_Z[df_Z["setup"] == setup].copy()
    best_cfg: Optional[DroopConfig] = None
    grid_rows: List[Dict] = []

    for lam in lambda_grid:
        for agg in agg_mode_grid:
            for p_q in p_grid:
                win_df = build_window_dataset_for_setup(
                    df_Z=df_setup,
                    cias_model=cias_model,
                    scenario_anom="DROOP",
                    W=window_size,
                    agg_mode=agg,
                    lambda_res=lam,
                    balance_per_workload=True,
                    seed=seed,
                )
                if win_df.empty or np.unique(win_df["label"]).size < 2:
                    continue

                fold_rows = evaluate_windows_kfold(
                    win_df=win_df,
                    setup=setup,
                    scenario_eval="DROOP",
                    W=window_size,
                    agg_mode=agg,
                    lambda_res=lam,
                    p_quantile=p_q,
                    n_splits=n_splits,
                    seed=seed,
                )
                if not fold_rows:
                    continue

                res_cfg = pd.DataFrame(fold_rows)
                res_cfg_all = res_cfg[res_cfg["workload"] == "ALL"]
                auc_mean = res_cfg_all["auc_roc"].mean()
                auc_pr_mean = res_cfg_all["auc_pr"].mean()
                f1_mean = res_cfg_all["f1"].mean()

                grid_rows.append(
                    {
                        "lambda_res": lam,
                        "agg_mode": agg,
                        "p_quantile": p_q,
                        "auc_roc_mean": auc_mean,
                        "auc_pr_mean": auc_pr_mean,
                        "f1_mean": f1_mean,
                    }
                )

                if (best_cfg is None) or (auc_mean > best_cfg.auc_roc_mean):
                    best_cfg = DroopConfig(
                        setup=setup,
                        window_size=window_size,
                        n_splits=n_splits,
                        lambda_res=lam,
                        agg_mode=agg,
                        p_quantile=p_q,
                        auc_roc_mean=auc_mean,
                        auc_pr_mean=auc_pr_mean,
                        f1_mean=f1_mean,
                    )

    if not grid_rows:
        print(f"[S7] No valid DROOP configs for setup {setup}; returning None.")
        return None

    grid_df = pd.DataFrame(grid_rows)
    droop_dir = OUT_ROOT / "DROOP_tuning"
    droop_dir.mkdir(parents=True, exist_ok=True)
    grid_csv = droop_dir / f"SETUP_{setup}_DROOP_tuning_grid.csv"
    grid_df.to_csv(grid_csv, index=False)

    if best_cfg is not None:
        best_cfg.csv_path = grid_csv
        print(
            f"[S7] Best DROOP config for setup {setup}: "
            f"λ_res={best_cfg.lambda_res:.3f}, agg_mode={best_cfg.agg_mode}, "
            f"p={best_cfg.p_quantile:.3f} → "
            f"AUC-ROC={best_cfg.auc_roc_mean:.3f}, "
            f"AUC-PR={best_cfg.auc_pr_mean:.3f}, "
            f"F1={best_cfg.f1_mean:.3f}"
        )
    else:
        print(f"[S7] No best DROOP config found for setup {setup}.")

    return best_cfg


# ------------------------------------------------
# S8: run EXACT evaluation for all scenarios
# ------------------------------------------------

def run_exact_eval_for_setup(
    setup: str,
    df_Z: pd.DataFrame,
    cias_model,
    scenarios_eval: List[str],
    window_sizes: List[int],
    n_splits_list: List[int],
    default_p_quantile: float,
    droop_cfg: Optional[DroopConfig],
    seed: int,
    out_dir: Path,
) -> pd.DataFrame:
    """
    Run full k-fold evaluation for a given setup across scenarios and window sizes.

    Uses DROOP-tuned (λ_res, agg_mode, p) when available for scenario='DROOP'.
    """
    df_setup = df_Z[df_Z["setup"] == setup].copy()
    scen_available = set(df_setup["scenario"].unique())

    all_rows: List[Dict] = []

    for scen in scenarios_eval:
        scen_u = scen.upper()
        if scen_u not in scen_available:
            print(
                f"[S8] Setup={setup}, scenario={scen_u} not present in telemetry; skipping."
            )
            continue

        print(f"\n[S8] === Setup {setup}, scenario={scen_u} ===")

        for W in window_sizes:
            for n_splits in n_splits_list:
                # Scenario-specific configuration
                if scen_u == "DROOP" and (droop_cfg is not None):
                    agg_mode = droop_cfg.agg_mode
                    p_quantile = droop_cfg.p_quantile
                    lambda_res = droop_cfg.lambda_res
                else:
                    # Default choices
                    agg_mode = "mean" if scen_u in ("DROOP", "SPECTRE") else "max"
                    p_quantile = default_p_quantile
                    lambda_res = 0.5

                win_df = build_window_dataset_for_setup(
                    df_Z=df_setup,
                    cias_model=cias_model,
                    scenario_anom=scen_u,
                    W=W,
                    agg_mode=agg_mode,
                    lambda_res=lambda_res,
                    balance_per_workload=True,
                    seed=seed,
                )

                if win_df.empty or np.unique(win_df["label"]).size < 2:
                    n_ben = int((win_df["label"] == 0).sum()) if not win_df.empty else 0
                    n_an = int((win_df["label"] == 1).sum()) if not win_df.empty else 0
                    print(
                        f"[S8] Setup={setup}, scenario={scen_u}, W={W}, agg={agg_mode}: "
                        f"n_ben_win={n_ben}, n_anom_win={n_an} → need both classes; skipping."
                    )
                    continue

                n_ben = int((win_df["label"] == 0).sum())
                n_an = int((win_df["label"] == 1).sum())
                print(
                    f"[S8] Setup={setup}, scenario={scen_u}, W={W}, agg={agg_mode}: "
                    f"n_ben_win={n_ben}, n_anom_win={n_an}"
                )

                fold_rows = evaluate_windows_kfold(
                    win_df=win_df,
                    setup=setup,
                    scenario_eval=scen_u,
                    W=W,
                    agg_mode=agg_mode,
                    lambda_res=lambda_res,
                    p_quantile=p_quantile,
                    n_splits=n_splits,
                    seed=seed,
                )
                all_rows.extend(fold_rows)

    res_df = pd.DataFrame(all_rows)
    if res_df.empty:
        print(f"[S8] No evaluation rows for setup={setup}.")
        return res_df

    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    csv_path = out_dir / f"SETUP_{setup}_EXACT_kfold_results.csv"
    res_df.to_csv(csv_path, index=False)
    print(
        f"\n[S8] Saved full K-fold + per-workload results for setup={setup} to:\n"
        f"  {csv_path}"
    )

    # Print quick summaries (workload='ALL')
    for scen in sorted(res_df["scenario"].unique()):
        sub = res_df[(res_df["workload"] == "ALL") & (res_df["scenario"] == scen)]
        if sub.empty:
            continue
        print(
            f"\n[S8] Summary (setup={setup}, scenario={scen}, workload='ALL', averaged over folds):"
        )
        for (W, k), g in sub.groupby(["window_size", "n_splits"]):
            auc_roc = g["auc_roc"].mean()
            auc_pr = g["auc_pr"].mean()
            f1 = g["f1"].mean()
            bal_acc = g["bal_acc"].mean()
            mcc = g["mcc"].mean()
            brier = g["brier"].mean()
            ece = g["ece"].mean()
            print(
                f"  {setup} | {scen:<8} | W={W:4d} | k={k:2d} | "
                f"AUC-ROC={auc_roc:.3f}, AUC-PR={auc_pr:.3f}, F1={f1:.3f}, "
                f"BalAcc={bal_acc:.3f}, MCC={mcc:.3f}, "
                f"Brier={brier:.4f}, ECE={ece:.4f}"
            )

    return res_df


# ------------------------------------------------
# S9: global / per-workload / fairness summaries
# ------------------------------------------------

def summarize_global_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Summarize metrics for workload='ALL' (across all workloads).
    """
    if "workload" not in df.columns:
        print("[PER-WL] No 'workload' column in result DF; cannot form global summary.")
        return pd.DataFrame()

    sub = df[df["workload"] == "ALL"].copy()
    if sub.empty:
        print('[PER-WL] No rows with workload="ALL"; cannot form global summary.')
        return pd.DataFrame()

    group_cols = ["setup", "scenario", "window_size", "n_splits"]
    agg = (
        sub.groupby(group_cols, as_index=False)
        .agg(
            n_test=("n_test", "mean"),
            n_benign=("n_benign", "mean"),
            n_anom=("n_anom", "mean"),
            auc_roc=("auc_roc", "mean"),
            auc_pr=("auc_pr", "mean"),
            f1=("f1", "mean"),
            bal_acc=("bal_acc", "mean"),
            mcc=("mcc", "mean"),
            brier=("brier", "mean"),
            ece=("ece", "mean"),
        )
    )
    return agg


def summarize_per_workload_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Summarize metrics per individual workload (excluding 'ALL').
    """
    if "workload" not in df.columns:
        print("[PER-WL] No 'workload' column in result DF; cannot form per-workload summary.")
        return pd.DataFrame()

    sub = df[df["workload"] != "ALL"].copy()
    if sub.empty:
        print('[PER-WL] Only workload="ALL" present; no per-workload metrics to summarize.')
        return pd.DataFrame()

    group_cols = ["setup", "scenario", "workload", "window_size", "n_splits"]
    agg = (
        sub.groupby(group_cols, as_index=False)
        .agg(
            n_test=("n_test", "mean"),
            n_benign=("n_benign", "mean"),
            n_anom=("n_anom", "mean"),
            auc_roc=("auc_roc", "mean"),
            auc_pr=("auc_pr", "mean"),
            f1=("f1", "mean"),
            bal_acc=("bal_acc", "mean"),
            mcc=("mcc", "mean"),
            brier=("brier", "mean"),
            ece=("ece", "mean"),
        )
    )
    return agg


def summarize_cross_workload_fairness(df: pd.DataFrame) -> pd.DataFrame:
    """
    For each (setup, scenario, window_size, n_splits), measure spread of metrics
    across workloads to quantify fairness/stability.
    """
    if "workload" not in df.columns:
        print("[FAIR] No 'workload' column in result DF; cannot compute fairness.")
        return pd.DataFrame()

    sub = df[df["workload"] != "ALL"].copy()
    if sub.empty:
        print("[FAIR] Only workload='ALL' present; no fairness metrics to compute.")
        return pd.DataFrame()

    group_cols = ["setup", "scenario", "window_size", "n_splits"]
    rows: List[Dict] = []

    for key, g in sub.groupby(group_cols):
        setup, scenario, W, k = key

        g_w = (
            g.groupby("workload", as_index=False)
            .agg(
                auc_roc=("auc_roc", "mean"),
                f1=("f1", "mean"),
                bal_acc=("bal_acc", "mean"),
            )
        )
        if g_w.empty:
            continue

        row = {
            "setup": setup,
            "scenario": scenario,
            "window_size": int(W),
            "n_splits": int(k),
            "n_workloads": int(g_w["workload"].nunique()),
            "auc_roc_min": float(g_w["auc_roc"].min()),
            "auc_roc_max": float(g_w["auc_roc"].max()),
            "auc_roc_range": float(g_w["auc_roc"].max() - g_w["auc_roc"].min()),
            "f1_min": float(g_w["f1"].min()),
            "f1_max": float(g_w["f1"].max()),
            "f1_range": float(g_w["f1"].max() - g_w["f1"].min()),
            "bal_acc_min": float(g_w["bal_acc"].min()),
            "bal_acc_max": float(g_w["bal_acc"].max()),
            "bal_acc_range": float(g_w["bal_acc"].max() - g_w["bal_acc"].min()),
        }
        rows.append(row)

    if not rows:
        print("[FAIR] No fairness rows generated.")
        return pd.DataFrame()

    return pd.DataFrame(rows)


def save_exact_summaries_for_setup(
    setup: str,
    res_df: pd.DataFrame,
    out_dir: Path,
):
    """
    Run global, per-workload, and fairness summaries for a setup
    and save all three CSVs.
    """
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    global_df = summarize_global_metrics(res_df)
    per_wl_df = summarize_per_workload_metrics(res_df)
    fair_df = summarize_cross_workload_fairness(res_df)

    # Global summary
    if global_df is not None and not global_df.empty:
        global_csv = out_dir / f"SETUP_{setup}_EXACT_summary_ALL.csv"
        global_df.to_csv(global_csv, index=False)
        print(f"[S9] Saved setup {setup} global summary to:\n  {global_csv}")
    else:
        print(f"[S9] No global summary rows for setup {setup}.")

    # Per-workload summary
    if per_wl_df is not None and not per_wl_df.empty:
        per_wl_csv = out_dir / f"SETUP_{setup}_EXACT_summary_per_workload.csv"
        per_wl_df.to_csv(per_wl_csv, index=False)
        print(f"[S9] Saved setup {setup} per-workload summary to:\n  {per_wl_csv}")
    else:
        print(f"[S9] No per-workload summary rows for setup {setup}.")

    # Fairness summary
    if fair_df is not None and not fair_df.empty:
        fair_csv = out_dir / f"SETUP_{setup}_EXACT_fairness.csv"
        fair_df.to_csv(fair_csv, index=False)
        print(f"[S9] Saved setup {setup} fairness summary to:\n  {fair_csv}")
    else:
        print(f"[S9] No fairness summary rows for setup {setup}.")

    return global_df, per_wl_df, fair_df


# ------------------------------------------------
# S_MASTER: driver
# ------------------------------------------------

def run_exact_with_droop_tuning():
    """
    Master driver:
      - tunes DROOP for A and B
      - runs full EXACT evaluation
      - saves global, per-workload, and fairness summaries
    Assumes:
      - OUT_ROOT defined
      - telemetry_A_Z, telemetry_B_Z defined
      - cias_A, cias_B defined (from S5)
    """
    print("\n[S_MASTER] Starting EXACT pipeline with DROOP tuning and full evaluation...")

    SCENARIOS_EVAL = ["DROOP", "RH", "SPECTRE"]
    WINDOW_SIZES = list(range(50, 1001, 50))  # 50,100,...,1000
    N_SPLITS_LIST = [5]
    RANDOM_SEED = 123

    # ---- Setup A ----
    print("\n[S_MASTER] Tuning DROOP for Setup A...")
    droop_cfg_A = tune_droop_for_setup(
        setup="A",
        df_Z=telemetry_A_Z,
        cias_model=cias_A,
        window_size=500,
        n_splits=5,
        seed=RANDOM_SEED,
    )

    print("\n[S_MASTER] Running EXACT evaluation for Setup A...")
    results_A = run_exact_eval_for_setup(
        setup="A",
        df_Z=telemetry_A_Z,
        cias_model=cias_A,
        scenarios_eval=SCENARIOS_EVAL,
        window_sizes=WINDOW_SIZES,
        n_splits_list=N_SPLITS_LIST,
        default_p_quantile=0.99,
        droop_cfg=droop_cfg_A,
        seed=RANDOM_SEED,
        out_dir=OUT_ROOT,
    )

    if not results_A.empty:
        save_exact_summaries_for_setup("A", results_A, OUT_ROOT)

    # ---- Setup B ----
    print("\n[S_MASTER] Tuning DROOP for Setup B...")
    droop_cfg_B = tune_droop_for_setup(
        setup="B",
        df_Z=telemetry_B_Z,
        cias_model=cias_B,
        window_size=500,
        n_splits=5,
        seed=RANDOM_SEED,
    )

    print("\n[S_MASTER] Running EXACT evaluation for Setup B...")
    results_B = run_exact_eval_for_setup(
        setup="B",
        df_Z=telemetry_B_Z,
        cias_model=cias_B,
        scenarios_eval=SCENARIOS_EVAL,
        window_sizes=WINDOW_SIZES,
        n_splits_list=N_SPLITS_LIST,
        default_p_quantile=0.99,
        droop_cfg=droop_cfg_B,
        seed=RANDOM_SEED,
        out_dir=OUT_ROOT,
    )

    if not results_B.empty:
        save_exact_summaries_for_setup("B", results_B, OUT_ROOT)

    print("\n[S_MASTER] Done.")


# To launch everything once S0–S5 are defined:
run_exact_with_droop_tuning()


In [ ]:
# ==============================
# S10 – EXACT CIAS energy & area
# ==============================
from pathlib import Path
import pandas as pd

# --- Paths (adjust if needed) ---
OUT_ROOT = Path("./results")
HW_CSV   = Path("./results")  # <-- your hw.csv

# ----------------------------------------------------
# 1) Load hardware macros for adder and multiplier
# ----------------------------------------------------
def load_hw_macros(hw_csv_path: Path) -> dict:
    """
    Load synthesized macros for adder and multiplier from hw.csv.

    Expected layout (your file matches this):
        metrics,add,mult
        area (mm^2),1165.2,4532
        power (mW),0.178,0.5146
        delay (ps),62.7,113.6
        cycles,3,4

    NOTE: The "area (mm^2)" numbers are ~1e3, which is too large for mm^2
    but reasonable for µm^2, so we interpret them as µm^2 and convert to mm^2.
    """
    df = pd.read_csv(hw_csv_path).set_index("metrics")

    AREA_SCALE = 1e-6  # interpret as µm^2 → mm^2
    area_add_mm2  = df.loc["area (mm^2)", "add"]  * AREA_SCALE
    area_mult_mm2 = df.loc["area (mm^2)", "mult"] * AREA_SCALE

    power_add_mW  = df.loc["power (mW)", "add"]
    power_mult_mW = df.loc["power (mW)", "mult"]

    delay_add_ps  = df.loc["delay (ps)", "add"]
    delay_mult_ps = df.loc["delay (ps)", "mult"]

    cycles_add    = df.loc["cycles", "add"]
    cycles_mult   = df.loc["cycles", "mult"]

    # Energy per operation in picojoules:
    #   E[J]  = P[W] * time[s]
    #   E[pJ] = power_mW * cycles * delay_ps * 1e-3
    E_add_pJ  = power_add_mW  * cycles_add  * delay_add_ps  * 1e-3
    E_mult_pJ = power_mult_mW * cycles_mult * delay_mult_ps * 1e-3

    return {
        "area_add_mm2":   area_add_mm2,
        "area_mult_mm2":  area_mult_mm2,
        "power_add_mW":   power_add_mW,
        "power_mult_mW":  power_mult_mW,
        "delay_add_ps":   delay_add_ps,
        "delay_mult_ps":  delay_mult_ps,
        "cycles_add":     cycles_add,
        "cycles_mult":    cycles_mult,
        "E_add_pJ":       E_add_pJ,
        "E_mult_pJ":      E_mult_pJ,
    }


# ----------------------------------------------------
# 2) Compute CIAS hardware costs per setup
# ----------------------------------------------------
def compute_exact_hw_costs_for_setups(
    hw_csv_path: Path,
    setup_config: dict,
    out_csv: Path | None = None,
) -> pd.DataFrame:
    """
    setup_config example:

    setup_config = {
        "A": {
            # number of physical units instantiated on chip
            "n_adders": 1,
            "n_multipliers": 1,

            # how many integer ops CIAS performs per window evaluation
            # (includes all E2, E1, and final score operations)
            "n_add_ops_per_eval":  80,
            "n_mult_ops_per_eval": 32,

            # CPU die area in mm^2 (for % of die)
            "die_area_mm2": 126.0,
        },
        ...
    }

    Returns a DataFrame with one row per setup.
    """
    macros = load_hw_macros(hw_csv_path)
    rows = []

    for setup, cfg in setup_config.items():
        n_adders   = int(cfg["n_adders"])
        n_mults    = int(cfg["n_multipliers"])
        n_add_ops  = int(cfg["n_add_ops_per_eval"])
        n_mult_ops = int(cfg["n_mult_ops_per_eval"])
        die_area   = cfg.get("die_area_mm2", None)

        # Total instantiated CIAS area (mm^2)
        area_mm2 = (
            n_adders * macros["area_add_mm2"]
            + n_mults * macros["area_mult_mm2"]
        )

        # Static power (when CIAS block is active)
        static_power_mW = (
            n_adders * macros["power_add_mW"]
            + n_mults * macros["power_mult_mW"]
        )

        # Dynamic energy per CIAS evaluation (per window)
        E_eval_pJ = (
            n_add_ops  * macros["E_add_pJ"]
            + n_mult_ops * macros["E_mult_pJ"]
        )
        E_eval_nJ = E_eval_pJ / 1e3

        frac_die = None
        if die_area is not None and die_area > 0:
            frac_die = 100.0 * area_mm2 / die_area

        rows.append(
            {
                "setup": setup,
                "n_adders": n_adders,
                "n_multipliers": n_mults,
                "n_add_ops_per_eval": n_add_ops,
                "n_mult_ops_per_eval": n_mult_ops,
                "area_total_mm2": area_mm2,
                "static_power_mW": static_power_mW,
                "energy_per_eval_pJ": E_eval_pJ,
                "energy_per_eval_nJ": E_eval_nJ,
                "die_area_mm2": die_area,
                "fraction_of_die_percent": frac_die,
            }
        )

    df = pd.DataFrame(rows)
    if out_csv is not None:
        out_csv.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(out_csv, index=False)
        print(f"[S10] Saved EXACT CIAS HW summary to:\n  {out_csv}")
    return df


# ----------------------------------------------------
# 3) CIAS configuration per setup (tie to scoring fn)
# ----------------------------------------------------
# Here we are encoding the CIAS datapath:
# - Same CIAS block for both setups (same number of ops per window).
# - 16 selected features, with ~5 adds and 2 multiplies per feature
#   across E2, E1, and the final score → 80 adds, 32 mults.
# - Die areas are approximate CPU die sizes in mm^2.

SETUP_HW_CONFIG = {
    "A": {
        "n_adders": 1,
        "n_multipliers": 1,
        "n_add_ops_per_eval":  80,   # CIAS integer adds per window
        "n_mult_ops_per_eval": 32,   # CIAS integer mults per window
        "die_area_mm2": 125.0,       # ~i5-7600K die area (mm^2)
    },
    "B": {
        "n_adders": 1,
        "n_multipliers": 1,
        "n_add_ops_per_eval":  80,
        "n_mult_ops_per_eval": 32,
        "die_area_mm2": 215.25,       # ~i5-12600K die area (mm^2)
    },
}

EXACT_HW_CSV = OUT_ROOT / "EXACT_CIAS_hw_summary.csv"
hw_summary_df = compute_exact_hw_costs_for_setups(
    HW_CSV,
    SETUP_HW_CONFIG,
    out_csv=EXACT_HW_CSV,
)

print("\n[S10] EXACT CIAS hardware cost summary:")
print(hw_summary_df.to_string(index=False))


---

In [ ]:
# ==============================
# S10. Causal structure & ranks
# ==============================

import numpy as np
import pandas as pd
from pathlib import Path

# For plotting; if networkx is missing, plotting will be skipped but CSVs still work
try:
    import networkx as nx
except ImportError:
    nx = None
import matplotlib.pyplot as plt


def _assign_feature_domain(feat: str) -> str:
    """
    Heuristic mapping from feature name to domain:
      - 'CORE'   : default, core performance counters
      - 'MEMORY' : DRAM / LLC / IMC traffic
      - 'SENSOR' : temperature / power / voltage sensors
    """
    name = feat.lower()
    if any(tok in name for tok in ["temp", "therm", "pk", "pkg", "pwr", "power", "volt", "vcc"]):
        return "SENSOR"
    if any(tok in name for tok in ["dram", "mem", "imc", "unc_m", "llc", "l3", "l2", "dimm"]):
        return "MEMORY"
    return "CORE"  # everything else treated as core performance counters


def _build_causal_edges_from_corr(
    corr_df: pd.DataFrame,
    corr_threshold: float = 0.35,
) -> pd.DataFrame:
    """
    Turn a correlation matrix into an edge list.
    Keep edges with |rho| >= corr_threshold.
    Edge direction follows CORE -> MEMORY -> SENSOR where possible.
    """
    features = list(corr_df.columns)
    domains = {f: _assign_feature_domain(f) for f in features}
    domain_order = {"CORE": 0, "MEMORY": 1, "SENSOR": 2}

    rows = []
    for i, f_i in enumerate(features):
        for j in range(i + 1, len(features)):
            f_j = features[j]
            rho = corr_df.iloc[i, j]
            if not np.isfinite(rho):
                continue
            if abs(rho) < corr_threshold:
                continue

            dom_i = domains[f_i]
            dom_j = domains[f_j]

            if domain_order[dom_i] < domain_order[dom_j]:
                src, dst = f_i, f_j
            elif domain_order[dom_j] < domain_order[dom_i]:
                src, dst = f_j, f_i
            else:
                # Same domain: orient by name just for consistency
                src, dst = (f_i, f_j) if f_i <= f_j else (f_j, f_i)

            rows.append(
                {
                    "src": src,
                    "dst": dst,
                    "rho": float(rho),
                    "abs_rho": float(abs(rho)),
                }
            )

    return pd.DataFrame(rows)


def _compute_graph_centrality(
    features: list[str],
    edges_df: pd.DataFrame,
) -> dict:
    """
    Simple degree centrality on the undirected version of the graph.
    """
    n = max(len(features) - 1, 1)
    deg_counts = {f: 0 for f in features}

    if edges_df is None or edges_df.empty:
        return {f: 0.0 for f in features}

    for row in edges_df.itertuples():
        deg_counts[row.src] += 1
        deg_counts[row.dst] += 1

    return {f: deg_counts[f] / n for f in features}


def _compute_feature_cias_correlation(
    df_Z: pd.DataFrame,
    features: list[str],
    score_col: str = "cias_sample_score",
    scenarios: tuple = ("DROOP", "RH", "SPECTRE"),
) -> dict:
    """
    For each feature, compute the average |Spearman rho| between that feature
    and the CIAS sample score over the specified anomaly scenarios.
    """
    corr_map = {f: [] for f in features}
    df_Z = df_Z.copy()
    df_Z["scenario"] = df_Z["scenario"].astype(str).str.upper()

    for scen in scenarios:
        scen_u = scen.upper()
        df_s = df_Z[df_Z["scenario"] == scen_u]
        if df_s.empty:
            continue

        for f in features:
            if f not in df_s.columns:
                continue
            sub = df_s[[f, score_col]].dropna()
            if sub[f].nunique() <= 1 or sub[score_col].nunique() <= 1:
                continue
            rho = sub.corr(method="spearman").iloc[0, 1]
            if np.isfinite(rho):
                corr_map[f].append(abs(float(rho)))

    # Average over scenarios (or 0 if never used)
    corr_avg = {}
    for f, vals in corr_map.items():
        corr_avg[f] = float(np.mean(vals)) if vals else 0.0
    return corr_avg


def build_causal_and_feature_rank_csvs_for_setup(
    setup: str,
    df_Z: pd.DataFrame,
    feature_cols: list[str],
    out_root: Path,
    corr_threshold: float = 0.35,
    top_k_plot: int = 20,
):
    """
    For a given setup ('A' or 'B'), build:
      - benign correlation-based causal edges
      - node table with domains and importance scores r_f
      - feature-rank CSV (sorted by importance)
    and (optionally) a combined figure with the causal graph + top-K bar chart.
    """
    setup = setup.upper()
    df_setup = df_Z[df_Z["setup"] == setup].copy()
    if df_setup.empty:
        raise ValueError(f"No rows found for setup={setup} in df_Z.")

    # 1) Benign-only correlation structure
    df_ben = df_setup[df_setup["scenario"].astype(str).str.upper() == "BENIGN"].copy()
    if df_ben.empty:
        raise ValueError(f"No BENIGN rows found for setup={setup}.")

    feats = [f for f in feature_cols if f in df_ben.columns]
    if not feats:
        raise ValueError(f"None of the requested feature_cols are in BENIGN df for setup={setup}.")

    X_ben = df_ben[feats].astype(float)
    corr_df = X_ben.corr(method="spearman")

    edges_df = _build_causal_edges_from_corr(corr_df, corr_threshold=corr_threshold)
    centrality = _compute_graph_centrality(feats, edges_df)
    cias_corr = _compute_feature_cias_correlation(
        df_setup, feats, score_col="cias_sample_score"
    )

    # 2) Importance scores \tilde{r}_f: graph centrality × CIAS-alignment
    graph_vec = np.array([centrality[f] for f in feats], dtype=float)
    corr_vec = np.array([cias_corr[f] for f in feats], dtype=float)

    # Normalize to [0,1] before combining
    g_max = max(graph_vec.max(), 1e-8)
    c_max = max(corr_vec.max(), 1e-8)
    graph_norm = graph_vec / g_max
    corr_norm = corr_vec / c_max

    r_raw = graph_norm * corr_norm
    r_max = max(r_raw.max(), 1e-8)
    r_norm = r_raw / r_max

    domains = {_f: _assign_feature_domain(_f) for _f in feats}

    nodes_rows = []
    ranks_rows = []
    for f, g_c, c_c, r in zip(feats, graph_norm, corr_norm, r_norm):
        nodes_rows.append(
            {
                "feature": f,
                "domain": domains[f],
                "graph_centrality": float(g_c),
                "cias_alignment": float(c_c),
                "importance_score": float(r),
            }
        )
        ranks_rows.append(
            {
                "feature": f,
                "domain": domains[f],
                "importance_score": float(r),
                "graph_centrality": float(g_c),
                "cias_alignment": float(c_c),
            }
        )

    nodes_df = pd.DataFrame(nodes_rows)
    ranks_df = pd.DataFrame(ranks_rows).sort_values(
        "importance_score", ascending=False
    ).reset_index(drop=True)

    # 3) Save CSVs
    causal_dir = Path(out_root) / "CAUSAL"
    causal_dir.mkdir(parents=True, exist_ok=True)

    nodes_csv = causal_dir / f"SETUP_{setup}_causal_nodes.csv"
    edges_csv = causal_dir / f"SETUP_{setup}_causal_edges.csv"
    ranks_csv = causal_dir / f"SETUP_{setup}_feature_ranks.csv"

    nodes_df.to_csv(nodes_csv, index=False)
    edges_df.to_csv(edges_csv, index=False)
    ranks_df.to_csv(ranks_csv, index=False)

    print(f"[CAUSAL] Saved nodes for setup {setup} to:   {nodes_csv}")
    print(f"[CAUSAL] Saved edges for setup {setup} to:   {edges_csv}")
    print(f"[CAUSAL] Saved ranks for setup {setup} to:   {ranks_csv}")

    # 4) Optional figure: causal graph + top-K bar chart
    try:
        if nx is None:
            print("[CAUSAL] networkx not installed; skipping causal graph plot.")
            return nodes_df, edges_df, ranks_df

        top_k = min(top_k_plot, len(ranks_df))
        top_feats = ranks_df.head(top_k)["feature"].tolist()
        top_scores = ranks_df.head(top_k)["importance_score"].to_numpy()
        top_domains = [domains[f] for f in top_feats]

        # Build graph for plotting (only features that appear in nodes_df; edges already filtered)
        G = nx.DiGraph()
        for row in nodes_rows:
            G.add_node(
                row["feature"],
                domain=row["domain"],
                importance=row["importance_score"],
            )

        for row in edges_df.itertuples():
            G.add_edge(row.src, row.dst, weight=row.abs_rho)

        # Layout: arrange by domain (y) and spread within each domain (x)
        domain_y = {"CORE": 0.0, "MEMORY": 1.0, "SENSOR": 2.0}
        pos = {}
        nodes_by_dom = {"CORE": [], "MEMORY": [], "SENSOR": []}
        for n, data in G.nodes(data=True):
            dom = data.get("domain", "CORE")
            nodes_by_dom.setdefault(dom, []).append(n)

        for dom, nodes_dom in nodes_by_dom.items():
            if not nodes_dom:
                continue
            xs = np.linspace(0.0, 1.0, len(nodes_dom))
            y = domain_y.get(dom, 0.0)
            for x, n in zip(xs, nodes_dom):
                pos[n] = (x, y)

        # Node colors by domain
        color_map = {"CORE": "#1f77b4", "MEMORY": "#ff7f0e", "SENSOR": "#2ca02c"}
        node_colors = [color_map.get(G.nodes[n].get("domain", "CORE"), "#1f77b4")
                       for n in G.nodes]
        node_sizes = [
            100 + 400 * G.nodes[n].get("importance", 0.0) for n in G.nodes
        ]
        edge_widths = []
        for _, _, d in G.edges(data=True):
            w = d.get("weight", 0.0) if isinstance(d, dict) else float(d)
            edge_widths.append(1.0 + 3.0 * w)

        fig, axes = plt.subplots(
            1, 2, figsize=(10, 4), gridspec_kw={"width_ratios": [2.0, 1.0]}
        )
        ax0, ax1 = axes

        # (a) causal graph
        nx.draw_networkx(
            G,
            pos=pos,
            ax=ax0,
            node_color=node_colors,
            node_size=node_sizes,
            width=edge_widths,
            with_labels=False,
            arrows=True,
            arrowstyle="-|>",
            arrowsize=10,
        )
        # Domain labels on the left
        for dom, y in domain_y.items():
            ax0.text(-0.1, y, dom, va="center", ha="right", fontsize=9)

        ax0.set_title(f"Setup {setup}: benign causal structure")
        ax0.set_axis_off()

        # (b) top-K bar chart
        y_pos = np.arange(top_k)
        bar_colors = [color_map.get(d, "#1f77b4") for d in top_domains]
        ax1.barh(y_pos, top_scores, color=bar_colors)
        ax1.set_yticks(y_pos)
        ax1.set_yticklabels(top_feats, fontsize=8)
        ax1.invert_yaxis()
        ax1.set_xlabel("Importance $\\tilde{r}_f$")
        ax1.set_title(f"Top-{top_k} influential features")

        fig.tight_layout()
        fig_path = causal_dir / f"SETUP_{setup}_causal_and_ranks.png"
        fig.savefig(fig_path, dpi=300)
        plt.close(fig)

        print(
            f"[CAUSAL] Saved combined causal + ranks figure for setup {setup} to:\n"
            f"  {fig_path}"
        )

    except Exception as e:
        print(f"[CAUSAL] WARNING: failed to plot causal figure for setup {setup}: {e}")

    return nodes_df, edges_df, ranks_df


In [ ]:
# Build causal CSVs + figures for both setups using the shared CIAS features
nodes_A, edges_A, ranks_A = build_causal_and_feature_rank_csvs_for_setup(
    setup="A",
    df_Z=telemetry_A_Z,
    feature_cols=shared_features,
    out_root=OUT_ROOT,
    corr_threshold=0.35,   # adjust if you want sparser / denser graph
    top_k_plot=20,         # how many bars to show in the right-hand subplot
)

nodes_B, edges_B, ranks_B = build_causal_and_feature_rank_csvs_for_setup(
    setup="B",
    df_Z=telemetry_B_Z,
    feature_cols=shared_features,
    out_root=OUT_ROOT,
    corr_threshold=0.35,
    top_k_plot=20,
)


In [ ]:
# =============================================================================
# S9 — Visualize the causal skeleton for 3 domains (Compute / Memory / Sensors)
#
# This version supports:
#   - Per-setup CSVs:
#       SETUP_A_causal_edges.csv / SETUP_A_causal_nodes.csv
#       SETUP_B_causal_edges.csv / SETUP_B_causal_nodes.csv
#   - Either:
#       results_dir / "csv" / <files>      (old layout), or
#       results_dir / <files>              (when results_dir == CAUSAL folder)
#
# Expected columns:
#   Edges CSV : src, dst, rho, abs_rho        (we use abs_rho as edge strength)
#               or u, v, strength, domain_u, domain_v (legacy)
#   Nodes CSV : feature, domain, ...
#               domain in {CORE, MEMORY, SENSOR} (mapped to compute/memory/sensors)
#
# Usage examples:
#   CAUSAL_DIR = Path("./results")
#   plot_three_domain_networks(CAUSAL_DIR, setup="A")
#   plot_three_domain_networks(CAUSAL_DIR, setup="B")
# =============================================================================
from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def plot_three_domain_networks(results_dir: Path,
                               setup=None,
                               label_only_top_n: int = 10,
                               spring_k_full: float = 10.0,
                               spring_k_domain: float = 5.0,
                               seed: int = 42,
                               show_labels_on_main_graph: bool = False):
    """
    Build and plot the 3-domain causal graph (compute / memory / sensors),
    optionally using per-setup edges/nodes CSVs.

    Parameters
    ----------
    results_dir : Path
        Root directory for a given EXACT run, or the CAUSAL folder itself.
        We look for CSVs in:
          - results_dir / "csv" / ..., if that subfolder exists; else
          - results_dir / ...
    setup : {"A","B",None}
        Which setup's CSVs to use. If "A" or "B", we look for
          SETUP_<setup>_causal_edges.csv
          SETUP_<setup>_causal_nodes.csv
        under the chosen CSV directory. If None, falls back to
          causal_skeleton_edges.csv (legacy).
    """
    import networkx as nx

    results_dir = Path(results_dir)

    # Detect whether results_dir already *is* the CAUSAL folder
    csv_subdir = results_dir / "csv"
    if csv_subdir.exists():
        csv_dir = csv_subdir
    else:
        csv_dir = results_dir

    fig_dir = results_dir / "figures"
    fig_dir.mkdir(parents=True, exist_ok=True)

    # -------------------------------------------------------------------------
    # 1) Pick edges and nodes files based on "setup"
    # -------------------------------------------------------------------------
    default_edges = csv_dir / "causal_skeleton_edges.csv"
    edges_p = default_edges
    nodes_p = None

    if setup is not None:
        setup = str(setup).upper()
        edges_candidate = csv_dir / f"SETUP_{setup}_causal_edges.csv"
        nodes_candidate = csv_dir / f"SETUP_{setup}_causal_nodes.csv"

        if edges_candidate.exists():
            edges_p = edges_candidate
            nodes_p = nodes_candidate if nodes_candidate.exists() else None
            print(f"[S9] Using per-setup edges file {edges_p.name} (setup={setup})")
            if nodes_p is None:
                print(f"[S9] WARNING: {nodes_candidate.name} not found; "
                      "domains will be inferred from names.")
        else:
            print(f"[S9] Per-setup edges CSV {edges_candidate} not found; "
                  f"falling back to {default_edges.name}")

    if not edges_p.exists():
        print(f"[S9] {edges_p} not found. Please make sure the causal edges CSV exists.")
        return

    edges_df = pd.read_csv(edges_p)

    nodes_df = None
    if nodes_p is not None and nodes_p.exists():
        nodes_df = pd.read_csv(nodes_p)
        print(f"[S9] Loaded node metadata from {nodes_p.name}")

    # -------------------------------------------------------------------------
    # 2) Standardize edge columns: u, v, strength
    # -------------------------------------------------------------------------
    # New CSVs use "src"/"dst"; old ones may already have "u"/"v".
    if "u" not in edges_df.columns and "src" in edges_df.columns:
        edges_df = edges_df.rename(columns={"src": "u", "dst": "v"})

    # Strength: use abs_rho if present, else |rho|, else 1.0 fallback.
    if "strength" not in edges_df.columns:
        if "abs_rho" in edges_df.columns:
            edges_df["strength"] = edges_df["abs_rho"].astype(float).abs()
        elif "rho" in edges_df.columns:
            edges_df["strength"] = edges_df["rho"].astype(float).abs()
        else:
            edges_df["strength"] = 1.0

    # -------------------------------------------------------------------------
    # 3) Map node -> domain using nodes CSV when available
    #     Nodes CSV has domain in {CORE, MEMORY, SENSOR}.
    # -------------------------------------------------------------------------
    feat_to_dom = {}
    if nodes_df is not None and "feature" in nodes_df.columns and "domain" in nodes_df.columns:
        dom_map = {
            "CORE": "compute",
            "COMPUTE": "compute",
            "CPU": "compute",
            "MEMORY": "memory",
            "MEM": "memory",
            "CACHE": "memory",
            "SENSOR": "sensors",
            "SENSORS": "sensors",
        }
        for _, row in nodes_df.iterrows():
            feat = str(row["feature"])
            dom_raw = str(row["domain"]).upper()
            dom = dom_map.get(dom_raw)

            # If it's some other label, fall back to name-based heuristics
            if dom is None:
                fl = feat.lower()
                if any(k in fl for k in ["temp", "power", "energy", "volt"]):
                    dom = "sensors"
                elif any(k in fl for k in ["l2", "l3", "mem"]):
                    dom = "memory"
                elif any(k in fl for k in ["core", "ipc", "inst", "cycle"]):
                    dom = "compute"
                else:
                    dom = "other"
            feat_to_dom[feat] = dom

    def _infer_domain_local(name: str) -> str:
        """Infer compute/memory/sensors/other using node CSV mapping if possible."""
        name = str(name)
        if name in feat_to_dom:
            return feat_to_dom[name]
        lname = name.lower()
        if any(k in lname for k in ["temp", "power", "energy", "volt"]):
            return "sensors"
        if any(k in lname for k in ["l2", "l3", "mem"]):
            return "memory"
        if any(k in lname for k in ["core", "ipc", "inst", "cycle"]):
            return "compute"
        return "other"

    # Ensure domain_u / domain_v exist and are normalized to our 3 domains
    if "domain_u" not in edges_df.columns or "domain_v" not in edges_df.columns:
        print("[S9] domain_u/domain_v missing — deriving from node CSV or name patterns.")
        edges_df["domain_u"] = edges_df["u"].map(_infer_domain_local)
        edges_df["domain_v"] = edges_df["v"].map(_infer_domain_local)
    else:
        # If they exist (legacy file), normalize to compute/memory/sensors
        def _norm_dom(val):
            if pd.isna(val):
                return "other"
            s = str(val).lower()
            if s in ["core", "cpu", "compute"]:
                return "compute"
            if s in ["mem", "memory", "cache"]:
                return "memory"
            if s in ["sensor", "sensors", "pvt", "thermal", "power"]:
                return "sensors"
            return s
        edges_df["domain_u"] = edges_df["domain_u"].map(_norm_dom)
        edges_df["domain_v"] = edges_df["domain_v"].map(_norm_dom)

    # -------------------------------------------------------------------------
    # 4) Normalize edge strength into [0,1] so color/width are comparable.
    # -------------------------------------------------------------------------
    s = edges_df["strength"].astype(float).replace([np.inf, -np.inf], np.nan).fillna(0.0).values
    if len(s) and np.nanmax(s) > np.nanmin(s):
        s_norm = (s - np.nanmin(s)) / (np.nanmax(s) - np.nanmin(s))
    else:
        s_norm = np.zeros_like(s)
    edges_df["strength_norm"] = s_norm

    # -------------------------------------------------------------------------
    # 5) Build an undirected graph: nodes = signals, edges = discovered links.
    # -------------------------------------------------------------------------
    G = nx.Graph()
    for _, r in edges_df.iterrows():
        u, v = str(r["u"]), str(r["v"])
        w = float(r["strength_norm"])
        du, dv = str(r["domain_u"]).lower(), str(r["domain_v"]).lower()
        G.add_node(u)
        G.add_node(v)
        G.add_edge(u, v, strength=w, domain_u=du, domain_v=dv)

    print(f"[S9] Full graph → nodes={G.number_of_nodes()} edges={G.number_of_edges()}")

    # -------------------------------------------------------------------------
    # 6) Assign a domain to each node via majority vote over incident edges.
    #     (We keep only compute / memory / sensors; everything else is "other".)
    # -------------------------------------------------------------------------
    domain_counts = {}
    for u, v, d in G.edges(data=True):
        for node, dn in [(u, d.get("domain_u", "other")), (v, d.get("domain_v", "other"))]:
            if node not in domain_counts:
                domain_counts[node] = {"compute": 0, "memory": 0, "sensors": 0, "other": 0}
            if dn in domain_counts[node]:
                domain_counts[node][dn] += 1
            else:
                domain_counts[node]["other"] += 1

    def pick_majority(dct):
        # Priority if there is a tie: compute > memory > sensors > other
        order = ["compute", "memory", "sensors", "other"]
        m = max(dct.values()) if dct else 0
        cands = [k for k, v in dct.items() if v == m]
        for k in order:
            if k in cands:
                return k
        return "other"

    node_domain = {n: pick_majority(domain_counts.get(n, {})) for n in G.nodes()}
    keep_nodes = [n for n, dn in node_domain.items() if dn in {"compute", "memory", "sensors"}]
    G3 = G.subgraph(keep_nodes).copy()
    print(f"[S9] 3-domain graph → nodes={G3.number_of_nodes()} edges={G3.number_of_edges()}")

    # Degree (number of neighbors) is used to scale node size and pick labels.
    deg = dict(G3.degree())
    if deg:
        dmax, dmin = max(deg.values()), min(deg.values())
    else:
        dmax = dmin = 1

    def scale_size(d):
        # Larger range for nicer PPT figures
        if dmax == dmin:
            return 1800.0
        return 1800.0 + 1500.0 * (d - dmin) / (dmax - dmin)

    # Pastel domain colors for PPT
    domain_colors = {
        "compute": "#9ecae1",  # light blue
        "memory":  "#a1d99b",  # light green
        "sensors": "#fdae6b",  # light orange
    }

    # VERY LARGE FONT SETTINGS FOR PRESENTATIONS
    rc = {
        "font.size": 36,
        "axes.titlesize": 44,
        "axes.labelsize": 40,
        "legend.fontsize": 38,
        "xtick.labelsize": 32,
        "ytick.labelsize": 32,
        "figure.titlesize": 48,
    }

    with plt.rc_context(rc):

        # =========================
        # (A) Full 3-domain network - NO FEATURE NAMES (clean presentation)
        # =========================
        if G3.number_of_nodes() > 0:
            print("[S9] Laying out full 3-domain graph (no feature names)...")
            pos = nx.spring_layout(G3, k=spring_k_full, iterations=200, seed=seed)

            node_sizes = [scale_size(deg[n]) for n in G3.nodes()]
            node_colors_seq = [
                domain_colors.get(node_domain.get(n, "compute"), "#9ecae1")
                for n in G3.nodes()
            ]
            edge_strengths = [G3[u][v].get("strength", 0.0) for u, v in G3.edges()]
            edge_colors = [plt.cm.viridis(x) for x in edge_strengths]
            edge_widths = [2.2 + 5.0 * x for x in edge_strengths]

            fig, ax = plt.subplots(figsize=(32, 28))
            ax.margins(0.10)

            # Draw edges first (so nodes appear on top)
            nx.draw_networkx_edges(
                G3, pos,
                edge_color=edge_colors,
                width=edge_widths,
                alpha=0.45,
                ax=ax,
            )

            # Draw nodes (larger and more visible since no labels)
            nx.draw_networkx_nodes(
                G3, pos,
                node_size=node_sizes,
                node_color=node_colors_seq,
                alpha=0.96,
                edgecolors="black",
                linewidths=2.2,
                ax=ax,
            )

            title_suffix = f" (Setup {setup})" if setup is not None else ""
            ax.set_title(f"High-Density Causal Skeleton{title_suffix}\n"
                         "(Compute / Memory / Sensors only)",
                         fontweight="bold", pad=28)
            ax.axis("off")

            # Legend outside plot so it never covers nodes
            from matplotlib.patches import Patch
            legend_elements = [
                Patch(facecolor=domain_colors["compute"],
                      label="Compute features",
                      alpha=0.95),
                Patch(facecolor=domain_colors["memory"],
                      label="Memory features",
                      alpha=0.95),
                Patch(facecolor=domain_colors["sensors"],
                      label="Sensor features",
                      alpha=0.95),
            ]
            ax.legend(
                handles=legend_elements,
                loc="upper left",
                bbox_to_anchor=(1.02, 1.0),
                borderaxespad=0.0,
                frameon=True,
                framealpha=0.95,
                fontsize=34,
            )

            # Edge-strength colorbar
            if edge_strengths:
                sm = plt.cm.ScalarMappable(
                    cmap=plt.cm.viridis,
                    norm=plt.Normalize(vmin=min(edge_strengths),
                                       vmax=max(edge_strengths)),
                )
                sm.set_array([])
                cbar = fig.colorbar(sm, ax=ax, shrink=0.85,
                                    aspect=30, pad=0.04)
                cbar.set_label("Connection strength (normalized)",
                               rotation=270, labelpad=32, fontsize=38)
                cbar.ax.tick_params(labelsize=32)

            plt.tight_layout()
            base_name = "high_density_causal_skeleton_no_other"
            if setup is not None:
                base_name = f"{base_name}_setup_{setup}"
            _safe_savefig(fig_dir / f"{base_name}.png", fig)
            print(f"💾 Saved → {fig_dir / f'{base_name}.png'}")

            # ===========================================
            # (A2) Optional: Same graph WITH feature names
            # ===========================================
            if show_labels_on_main_graph:
                print("[S9] Creating alternative version WITH feature names...")
                fig2, ax2 = plt.subplots(figsize=(32, 28))
                ax2.margins(0.10)

                # Draw everything again
                nx.draw_networkx_edges(
                    G3, pos,
                    edge_color=edge_colors,
                    width=edge_widths,
                    alpha=0.45,
                    ax=ax2,
                )
                nx.draw_networkx_nodes(
                    G3, pos,
                    node_size=[s * 0.9 for s in node_sizes],  # Slightly smaller for labels
                    node_color=node_colors_seq,
                    alpha=0.96,
                    edgecolors="black",
                    linewidths=2.2,
                    ax=ax2,
                )

                # Add labels for top nodes only
                top_nodes = [n for n, _ in sorted(deg.items(),
                                                  key=lambda t: t[1],
                                                  reverse=True)[:label_only_top_n]]

                def clean_name(n):
                    if "Core" in n and "Socket" in n:
                        parts = n.split('.')
                        try:
                            core_num = n.split('(')[1].split()[0]
                            metric = parts[1].split('_')[-1] if len(parts) > 1 else "M"
                            return f"C{core_num}.{metric}"
                        except Exception:
                            return n[:15] + "..." if len(n) > 15 else n
                    if "System" in n:
                        parts = n.split('.')
                        return f"Sys.{parts[1]}" if len(parts) > 1 else "System"
                    if "Socket" in n and "Core" not in n:
                        parts = n.split('.')
                        return f"SKT.{parts[1]}" if len(parts) > 1 else "Socket"
                    # General shortening for other nodes
                    if len(n) > 20:
                        return n[:18] + "..."
                    return n

                labels = {n: clean_name(n) for n in top_nodes}
                nx.draw_networkx_labels(
                    G3,
                    {n: pos[n] for n in top_nodes},
                    labels,
                    font_size=24,
                    font_weight="bold",
                    ax=ax2,
                    bbox=dict(facecolor="white", edgecolor="none", alpha=0.85, pad=0.20),
                )

                ax2.set_title(
                    f"High-Density Causal Skeleton (With Labels){title_suffix}\n"
                    "(Compute / Memory / Sensors only)",
                    fontweight="bold", pad=28,
                )
                ax2.axis("off")

                # Add legend
                ax2.legend(
                    handles=legend_elements,
                    loc="upper left",
                    bbox_to_anchor=(1.02, 1.0),
                    borderaxespad=0.0,
                    frameon=True,
                    framealpha=0.95,
                    fontsize=34,
                )

                # Add colorbar
                if edge_strengths:
                    sm2 = plt.cm.ScalarMappable(
                        cmap=plt.cm.viridis,
                        norm=plt.Normalize(vmin=min(edge_strengths),
                                           vmax=max(edge_strengths)),
                    )
                    sm2.set_array([])
                    cbar2 = fig2.colorbar(sm2, ax=ax2, shrink=0.85,
                                          aspect=30, pad=0.04)
                    cbar2.set_label("Connection strength (normalized)",
                                    rotation=270, labelpad=32, fontsize=38)
                    cbar2.ax.tick_params(labelsize=32)

                plt.tight_layout()
                base_name2 = "high_density_causal_skeleton_with_labels"
                if setup is not None:
                    base_name2 = f"{base_name2}_setup_{setup}"
                _safe_savefig(fig_dir / f"{base_name2}.png", fig2)
                print(f"💾 Saved (with labels) → {fig_dir / f'{base_name2}.png'}")

        else:
            print("[S9] 3-domain graph is empty; skipping full plot.")

        # ==================================
        # (B) Domain-specific subgraphs (3x)
        # ==================================
        domains = ["compute", "memory", "sensors"]

        # ---- Combined 3-panel figure ----
        fig_comb, axes = plt.subplots(1, 3, figsize=(36, 14))
        axes = axes.flatten()

        for i, dom in enumerate(domains):
            dn_nodes = [n for n in G3.nodes() if node_domain.get(n) == dom]
            ax = axes[i]
            ax.axis("off")
            ax.set_title(f"{dom.capitalize()} domain ({len(dn_nodes)} nodes)",
                         fontsize=40, fontweight="bold", pad=20)
            if len(dn_nodes) < 2:
                continue

            H = G3.subgraph(dn_nodes).copy()
            pos_sub = nx.spring_layout(H, k=spring_k_domain, iterations=150, seed=seed)

            node_sizes_sub = [scale_size(deg.get(n, 1)) for n in H.nodes()]
            nx.draw_networkx_nodes(
                H, pos_sub,
                node_size=node_sizes_sub,
                node_color=domain_colors[dom],
                alpha=0.96,
                edgecolors="black",
                linewidths=2.0,
                ax=ax,
            )

            dom_edges = list(H.edges())
            if dom_edges:
                es = [H[u][v].get("strength", 0.0) for u, v in dom_edges]
                ec = [plt.cm.viridis(x) for x in es]
                ew = [1.8 + 4.0 * x for x in es]
                nx.draw_networkx_edges(
                    H, pos_sub,
                    edgelist=dom_edges,
                    edge_color=ec,
                    width=ew,
                    alpha=0.55,
                    ax=ax,
                )

            dom_deg = dict(H.degree())
            dom_top = [n for n, _ in sorted(dom_deg.items(),
                                            key=lambda t: t[1],
                                            reverse=True)[:max(3, label_only_top_n // 3)]]
            nx.draw_networkx_labels(
                H, pos_sub,
                {n: n[:15] + "..." if len(n) > 15 else n for n in dom_top},
                font_size=24,
                font_weight="bold",
                ax=ax,
                bbox=dict(facecolor="white", edgecolor="none", alpha=0.85, pad=0.18),
            )

        sup_title_suffix = f" (Setup {setup})" if setup is not None else ""
        plt.suptitle(f"Causal Skeleton — Domain-Specific Views{sup_title_suffix}",
                     fontsize=44, fontweight="bold", y=0.98)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        combo_name = "three_domain_networks_high_density"
        if setup is not None:
            combo_name = f"{combo_name}_setup_{setup}"
        _safe_savefig(fig_dir / f"{combo_name}.png", fig_comb)
        print(f"💾 Saved → {fig_dir / f'{combo_name}.png'}")

        # ---- Separate PNG per domain (for PPT) ----
        for dom in domains:
            dn_nodes = [n for n in G3.nodes() if node_domain.get(n) == dom]
            if len(dn_nodes) < 2:
                continue

            H = G3.subgraph(dn_nodes).copy()
            pos_sub = nx.spring_layout(H, k=spring_k_domain, iterations=150, seed=seed)

            fig_d, ax_d = plt.subplots(figsize=(22, 18))
            ax_d.axis("off")
            ax_d.set_title(f"{dom.capitalize()} domain ({len(dn_nodes)} nodes){sup_title_suffix}",
                           fontsize=44, fontweight="bold", pad=22)

            node_sizes_sub = [scale_size(deg.get(n, 1)) for n in H.nodes()]
            nx.draw_networkx_nodes(
                H, pos_sub,
                node_size=node_sizes_sub,
                node_color=domain_colors[dom],
                alpha=0.96,
                edgecolors="black",
                linewidths=2.2,
                ax=ax_d,
            )

            dom_edges = list(H.edges())
            if dom_edges:
                es = [H[u][v].get("strength", 0.0) for u, v in dom_edges]
                ec = [plt.cm.viridis(x) for x in es]
                ew = [2.0 + 4.5 * x for x in es]
                nx.draw_networkx_edges(
                    H, pos_sub,
                    edgelist=dom_edges,
                    edge_color=ec,
                    width=ew,
                    alpha=0.55,
                    ax=ax_d,
                )

            dom_deg = dict(H.degree())
            dom_top = [n for n, _ in sorted(dom_deg.items(),
                                            key=lambda t: t[1],
                                            reverse=True)[:max(4, label_only_top_n // 2)]]
            nx.draw_networkx_labels(
                H, pos_sub,
                {n: n[:12] + "..." if len(n) > 12 else n for n in dom_top},
                font_size=26,
                font_weight="bold",
                ax=ax_d,
                bbox=dict(facecolor="white", edgecolor="none", alpha=0.9, pad=0.18),
            )

            plt.tight_layout()
            out_dom = fig_dir / f"domain_{dom}_network_high_density"
            if setup is not None:
                out_dom = out_dom.with_name(out_dom.name + f"_setup_{setup}.png")
            else:
                out_dom = out_dom.with_suffix(".png")
            _safe_savefig(out_dom, fig_d)
            print(f"💾 Saved → {out_dom}")

        # =======================================
        # (C) Core-focused circular visualization
        #     Zoom in on per-core compute relationships.
        # =======================================
        core_nodes = [n for n in G3.nodes() if ("Core" in n and "Socket" in n)]
        Hcore = G3.subgraph(core_nodes).copy()
        print(f"[S9] Core subgraph → nodes={Hcore.number_of_nodes()} edges={Hcore.number_of_edges()}")

        if Hcore.number_of_nodes() > 0:
            pos_core = nx.circular_layout(Hcore)
            node_sizes_core = [scale_size(deg.get(n, 1)) for n in Hcore.nodes()]
            node_colors_core = [
                domain_colors.get(node_domain.get(n, "compute"), "#9ecae1")
                for n in Hcore.nodes()
            ]

            fig3, ax3 = plt.subplots(figsize=(30, 26))
            ax3.margins(0.15)

            nx.draw_networkx_nodes(
                Hcore, pos_core,
                node_size=node_sizes_core,
                node_color=node_colors_core,
                alpha=0.96,
                edgecolors="black",
                linewidths=2.2,
                ax=ax3,
            )
            core_edges = list(Hcore.edges())
            if core_edges:
                es = [Hcore[u][v].get("strength", 0.0) for u, v in core_edges]
                ec = [plt.cm.viridis(x) for x in es]
                ew = [1.6 + 3.5 * x for x in es]
                nx.draw_networkx_edges(
                    Hcore, pos_core,
                    edgelist=core_edges,
                    edge_color=ec,
                    width=ew,
                    alpha=0.45,
                    ax=ax3,
                )

            # Compact labels: C#.<metric>
            labels_core = {}
            for n in Hcore.nodes():
                try:
                    core_num = n.split('(')[1].split()[0]
                    metric = (n.split('.', 1)[1].split('_')[-1]) if ('.' in n) else "M"
                    labels_core[n] = f"C{core_num}.{metric}"
                except Exception:
                    labels_core[n] = n[:10] + "..." if len(n) > 10 else n

            nx.draw_networkx_labels(
                Hcore, pos_core,
                labels_core,
                font_size=26,
                font_weight="bold",
                ax=ax3,
                bbox=dict(facecolor="white", edgecolor="none", alpha=0.9, pad=0.18),
            )

            title = "Core-Level Causal Relationships (Compute)"
            if setup is not None:
                title += f" — Setup {setup}"
            ax3.set_title(title, fontsize=44, fontweight="bold", pad=24)
            ax3.axis("off")

            if core_edges:
                sm3 = plt.cm.ScalarMappable(
                    cmap=plt.cm.viridis,
                    norm=plt.Normalize(vmin=min(es), vmax=max(es)),
                )
                sm3.set_array([])
                cbar3 = fig3.colorbar(sm3, ax=ax3, shrink=0.80,
                                      aspect=30, pad=0.04)
                cbar3.set_label("Connection strength (normalized)",
                                rotation=270, labelpad=32, fontsize=38)
                cbar3.ax.tick_params(labelsize=32)

            plt.tight_layout()
            core_name = "core_network_circular_high_density"
            if setup is not None:
                core_name = f"{core_name}_setup_{setup}"
            _safe_savefig(fig_dir / f"{core_name}.png", fig3)
            print(f"💾 Saved → {fig_dir / f'{core_name}.png'}")

    # ================
    # Textual summary
    # ================
    dom_counts = {"compute": 0, "memory": 0, "sensors": 0}
    for n, d in node_domain.items():
        if n in G3 and d in dom_counts:
            dom_counts[d] += 1

    deg_sorted = sorted(deg.items(), key=lambda t: t[1], reverse=True)
    summary_name = "three_domain_summary.txt" if setup is None else f"three_domain_summary_setup_{setup}.txt"
    summary_txt = fig_dir / summary_name
    with open(summary_txt, "w") as f:
        f.write("HIGH-DENSITY 3-DOMAIN SKELETON SUMMARY\n")
        f.write(f"nodes={G3.number_of_nodes()} edges={G3.number_of_edges()}\n")
        f.write("domain_counts=" + json.dumps(dom_counts) + "\n")
        f.write("top_degree_nodes:\n")
        for n, d in deg_sorted[:20]:
            f.write(f"  {n}  deg={d}  domain={node_domain.get(n, '?')}\n")
    print(f"📝 Summary saved → {summary_txt.name}")


def run_S9_for(out: dict, setup=None):
    """
    Small helper: call S9 after S8 using the `out` dict that holds results_dir.
    If setup is not given, will try out.get("setup") or out.get("setup_label").
    """
    setup_use = setup or out.get("setup") or out.get("setup_label")
    return plot_three_domain_networks(out["results_dir"], setup=setup_use)


# Helper function for saving figures (add if missing)
def _safe_savefig(path, fig, dpi=150, bbox_inches='tight'):
    """Save figure with error handling."""
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    try:
        fig.savefig(path, dpi=dpi, bbox_inches=bbox_inches)
    except Exception as e:
        print(f"Warning: Could not save {path}: {e}")
        # Try with lower DPI
        try:
            fig.savefig(path, dpi=100, bbox_inches=bbox_inches)
        except Exception:
            pass


In [ ]:
from pathlib import Path
CAUSAL_DIR = Path("./results")

plot_three_domain_networks(CAUSAL_DIR, setup="A")
plot_three_domain_networks(CAUSAL_DIR, setup="B")



### Trade-off: Window size vs. Accuracy

In [ ]:
# ============================================
# S11 – Window-size trade-off: bal_acc & MCC
# ============================================
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

OUT_ROOT = Path("./results")  # same as S0/S6

def load_balacc_mcc_vs_W(setup: str,
                         scenarios=("DROOP", "RH", "SPECTRE"),
                         out_root: Path = OUT_ROOT) -> pd.DataFrame:
    """
    Load per-workload summary and average balanced accuracy (bal_acc)
    and MCC over workloads and anomaly scenarios for each window size W.
    """
    per_wl_csv = out_root / f"SETUP_{setup}_EXACT_summary_per_workload.csv"
    df = pd.read_csv(per_wl_csv)

    # Keep only requested anomaly scenarios (drop BENIGN rows if present)
    if "scenario" in df.columns and scenarios is not None:
        df = df[df["scenario"].isin(scenarios)]

    # Group by window size (and optionally n_splits if you want)
    grp_cols = ["window_size"]
    agg = (
        df.groupby(grp_cols, as_index=False)
          .agg(
              bal_acc=("bal_acc", "mean"),
              mcc=("mcc", "mean"),
              n_test=("n_test", "mean"),
          )
          .sort_values("window_size")
    )
    return agg


def find_breakpoint(df: pd.DataFrame,
                    metric: str,
                    threshold: float = 0.99) -> int | None:
    """
    Return the smallest window_size where the given metric drops below
    'threshold'. If this never happens, return None.
    """
    df = df.sort_values("window_size")
    bad = df[df[metric] < threshold]
    if not bad.empty:
        return int(bad["window_size"].iloc[0])
    return None


# ---- Load curves for both setups ----
balacc_mcc_A = load_balacc_mcc_vs_W("A")
balacc_mcc_B = load_balacc_mcc_vs_W("B")

print("Setup A – balanced accuracy & MCC vs window size:")
print(balacc_mcc_A.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

print("\nSetup B – balanced accuracy & MCC vs window size:")
print(balacc_mcc_B.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

# ---- Find where things 'start going bad' (example: < 0.99) ----
thr = 0.99  # you can change to 0.98 or 0.95 if you prefer

for setup, df in [("A", balacc_mcc_A), ("B", balacc_mcc_B)]:
    wb_bal = find_breakpoint(df, "bal_acc", threshold=thr)
    wb_mcc = find_breakpoint(df, "mcc", threshold=thr)

    print(f"\n[Setup {setup}] first window size with bal_acc < {thr}: {wb_bal}")
    print(f"[Setup {setup}] first window size with MCC     < {thr}: {wb_mcc}")

# ---- Plot: two panels, bal_acc and MCC vs W ----
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharex=True)

ax1, ax2 = axes

# Balanced accuracy panel
ax1.plot(balacc_mcc_A["window_size"], balacc_mcc_A["bal_acc"],
         marker="o", linestyle="-", label="Setup A")
ax1.plot(balacc_mcc_B["window_size"], balacc_mcc_B["bal_acc"],
         marker="s", linestyle="--", label="Setup B")
ax1.set_xlabel("Window size $W$")
ax1.set_ylabel("Balanced accuracy")
ax1.set_title("Balanced accuracy vs. window size")
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0.0, 1.01)

# MCC panel
ax2.plot(balacc_mcc_A["window_size"], balacc_mcc_A["mcc"],
         marker="o", linestyle="-", label="Setup A")
ax2.plot(balacc_mcc_B["window_size"], balacc_mcc_B["mcc"],
         marker="s", linestyle="--", label="Setup B")
ax2.set_xlabel("Window size $W$")
ax2.set_ylabel("MCC")
ax2.set_title("MCC vs. window size")
ax2.grid(True, alpha=0.3)
ax2.set_ylim(-0.05, 1.01)

# Shared legend
handles, labels = ax2.get_legend_handles_labels()
fig.legend(handles, labels, loc="lower center", ncol=2, bbox_to_anchor=(0.5, -0.02))

fig.tight_layout()
plt.subplots_adjust(bottom=0.20)

out_fig = OUT_ROOT / "EXACT_balacc_mcc_vs_W_50_to_1000.png"
plt.savefig(out_fig, dpi=300)
print(f"\nSaved figure to: {out_fig}")
plt.show()


# **Hardware comparison

In [ ]:
import numpy as np
import pandas as pd

# ============================================================
# 1) Baselines and constants
# ============================================================

# Setup B die area (mm^2)
SETUP_B_DIE_AREA_MM2 = 215.25

# Workload powers on DDR5 platform (W) – non-idle
POWER_WORKLOAD = {
    "DFT": 139.5,
    "DJ": 64.7,
    "DP": 99.3,
    "GS": 96.55,
    "GL": 69.1,
    "HA": 87.5,
    "JA": 89.8,
    "MM": 141.5,
    "NI": 97.5,
    "OE": 114.7,
    "PI": 96.8,
    "SH": 113.0,
    "TR": 94.9,
}

workload_powers = np.array(list(POWER_WORKLOAD.values()))
median_workload_power_W = np.median(workload_powers)

# Idle power baseline (W) – from your earlier comment
IDLE_POWER_W = 35.5

# ------------------------------------------------------------
# EXACT (CIAS) parameters
# ------------------------------------------------------------
EXACT_AREA_MM2 = 0.0057
EXACT_AREA_OVERHEAD_PCT = 100.0 * EXACT_AREA_MM2 / SETUP_B_DIE_AREA_MM2

# Given: 0.0032% median overhead relative to workload power at 4.5 GHz
EXACT_WORKLOAD_OVERHEAD_PCT = 0.0032

# Derive CIAS power in W from that percentage and the median workload power
cias_power_W = (EXACT_WORKLOAD_OVERHEAD_PCT / 100.0) * median_workload_power_W

# Compute idle overhead by comparing same CIAS power against idle baseline
EXACT_IDLE_OVERHEAD_PCT = (cias_power_W / IDLE_POWER_W) * 100.0

# ============================================================
# 2) Build comparison table (EXACT vs OCTANE vs E-SCOUT)
# ============================================================

data = [
    {
        "method": "EXACT (CIAS)",
        "n_features": 15,
        "area_overhead_pct": EXACT_AREA_OVERHEAD_PCT,
        "power_idle_overhead_pct": EXACT_IDLE_OVERHEAD_PCT,
        "power_workload_overhead_pct": EXACT_WORKLOAD_OVERHEAD_PCT,
    },
    {
        "method": "OCTANE",
        "n_features": 227,           # ~active subset on Setup B
        "area_overhead_pct": 1.2,    # paper
        "power_idle_overhead_pct": 2.6,
        "power_workload_overhead_pct": 0.9,  # avg during workloads
    },
    {
        "method": "E-SCOUT",
        "n_features": 230,           # top 50% of 460 features
        "area_overhead_pct": 2.2,
        "power_idle_overhead_pct": 1.0,      # idle overhead
        "power_workload_overhead_pct": np.nan,  # not explicitly reported
    },
]

df = pd.DataFrame(data)

# ============================================================
# 3) Print a clean table (no figure generation)
# ============================================================

# Add a simple derived ratio (optional, but often useful)
df["area_to_idle_power_ratio"] = df["area_overhead_pct"] / df["power_idle_overhead_pct"].replace(0, np.nan)

# Friendly formatting for console output
df_print = df.copy()

# Format numeric columns as strings with units
df_print["area_overhead_pct"] = df_print["area_overhead_pct"].map(lambda x: f"{x:.6f}%")
df_print["power_idle_overhead_pct"] = df_print["power_idle_overhead_pct"].map(lambda x: f"{x:.6f}%")
df_print["power_workload_overhead_pct"] = df_print["power_workload_overhead_pct"].map(
    lambda x: "N/A" if pd.isna(x) else f"{x:.6f}%"
)
df_print["area_to_idle_power_ratio"] = df_print["area_to_idle_power_ratio"].map(
    lambda x: "N/A" if pd.isna(x) else f"{x:.3f}"
)

# Reorder columns for readability
df_print = df_print[
    [
        "method",
        "n_features",
        "area_overhead_pct",
        "power_idle_overhead_pct",
        "power_workload_overhead_pct",
        "area_to_idle_power_ratio",
    ]
]

print("\n" + "=" * 110)
print("HARDWARE OVERHEAD COMPARISON SUMMARY (TABLE ONLY)")
print("=" * 110)
print(df_print.to_string(index=False))
print("=" * 110)

# ============================================================
# 4) Optional: save to CSV for Overleaf / paper tables
# ============================================================

out_csv = "hardware_overhead_comparison_table.csv"
df.to_csv(out_csv, index=False)
print(f"\nSaved table to: {out_csv}")

# ============================================================
# 5) Optional: short key findings (text only)
# ============================================================

exact_vs_octane_area = df.loc[df["method"] == "OCTANE", "area_overhead_pct"].values[0] / df.loc[
    df["method"] == "EXACT (CIAS)", "area_overhead_pct"
].values[0]
exact_vs_escout_area = df.loc[df["method"] == "E-SCOUT", "area_overhead_pct"].values[0] / df.loc[
    df["method"] == "EXACT (CIAS)", "area_overhead_pct"
].values[0]

print("\n" + "=" * 110)
print("KEY FINDINGS (TEXT ONLY)")
print("=" * 110)
print(f"• EXACT (CIAS) area overhead: {EXACT_AREA_OVERHEAD_PCT:.6f}% (≈{exact_vs_octane_area:,.0f}× lower than OCTANE; ≈{exact_vs_escout_area:,.0f}× lower than E-SCOUT).")
print(f"• EXACT (CIAS) idle power overhead (derived from workload %): {EXACT_IDLE_OVERHEAD_PCT:.6f}%.")
print(f"• OCTANE: {df.loc[1,'n_features']} features, {df.loc[1,'area_overhead_pct']:.2f}% area, {df.loc[1,'power_idle_overhead_pct']:.2f}% idle power.")
print(f"• E-SCOUT: {df.loc[2,'n_features']} features, {df.loc[2,'area_overhead_pct']:.2f}% area, {df.loc[2,'power_idle_overhead_pct']:.2f}% idle power (workload power overhead not reported).")
print("=" * 110)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data
setups = ["A", "B"]
cias_area_mm2 = 0.0030499999999999998  # Same for both setups
cpu_die_mm2 = {"A": 125.0, "B": 215.25}
cias_cpu_die_percent = {
    "A": 0.0024399999999999995,
    "B": 0.0014169570267131243,
}

# Convert to lists
percents = [cias_cpu_die_percent[s] for s in setups]

# High-quality styling for publication
plt.rcParams.update({
    "font.size": 14,
    "axes.titlesize": 16,
    "axes.labelsize": 14,
    "xtick.labelsize": 13,
    "ytick.labelsize": 13,
    "font.family": "DejaVu Sans",
    "figure.dpi": 300,  # High resolution
})

# Create the figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), constrained_layout=True)

# Professional color scheme
color_cias = '#2E86AB'  # Professional blue
color_percent = '#A23B72'  # Professional purple
color_highlight = '#F18F01'  # Accent color

# -------------------------------
# Left plot: CIAS Area (single bar)
# -------------------------------
bars_left = ax1.bar(['CIAS Area'], [cias_area_mm2], 
                   color=color_cias, alpha=0.8, width=0.6,
                   edgecolor='black', linewidth=1.2)

ax1.set_ylabel('Area (mm²)', fontweight='bold', color=color_cias)
ax1.set_title('Absolute CIAS Area\n(Same for Both Setups)', 
              fontweight='bold', pad=20)

# Value annotation
ax1.text(0, cias_area_mm2 * 1.15, f'{cias_area_mm2:.6f} mm²', 
         ha='center', va='bottom', fontsize=13, fontweight='bold',
         color=color_cias, bbox=dict(boxstyle="round,pad=0.3", 
                                   facecolor='white', 
                                   edgecolor=color_cias, 
                                   alpha=0.9))

ax1.grid(axis='y', linestyle='--', alpha=0.4, color='gray')
ax1.set_ylim(0, cias_area_mm2 * 1.8)

# Clean spines
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

# -------------------------------
# Right plot: Percentage Share
# -------------------------------
x_pos = np.arange(len(setups))
bars_right = ax2.bar(x_pos, percents, 
                    color=color_percent, alpha=0.8, width=0.6,
                    edgecolor='black', linewidth=1.2)

ax2.set_xlabel('Setup', fontweight='bold')
ax2.set_ylabel('Share of CPU Die Area (%)', fontweight='bold', color=color_percent)
ax2.set_xticks(x_pos)
ax2.set_xticklabels([f'Setup {s}' for s in setups])
ax2.set_title('CIAS Die Area Overhead', fontweight='bold', pad=20)

# Value annotations with percentage formatting
for i, (rect, val, setup) in enumerate(zip(bars_right, percents, setups)):
    height = rect.get_height()
    die_size = cpu_die_mm2[setup]
    
    # Main percentage value
    ax2.text(rect.get_x() + rect.get_width()/2., height * 1.25,
             f'{val:.4%}',  # Format as percentage
             ha='center', va='bottom', fontsize=13, fontweight='bold',
             color=color_percent,
             bbox=dict(boxstyle="round,pad=0.3", 
                     facecolor='white', 
                     edgecolor=color_percent, 
                     alpha=0.9))
    
    # CPU die size info
    ax2.text(rect.get_x() + rect.get_width()/2., -max(percents) * 0.15,
             f'CPU Die: {die_size} mm²',
             ha='center', va='top', fontsize=11, style='italic',
             color='dimgray')

ax2.grid(axis='y', linestyle='--', alpha=0.4, color='gray')
ax2.set_ylim(-max(percents) * 0.25, max(percents) * 1.6)

# Clean spines
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

# Overall title
plt.suptitle('CIAS Area Analysis: Absolute Size and Relative Overhead', 
             fontsize=18, fontweight='bold', y=1.02)

# Explanatory note
fig.text(0.5, 0.01, 
         'Note: CIAS area is identical in both setups (0.00305 mm²), but represents different overhead percentages due to different CPU die sizes.',
         ha='center', fontsize=11, style='italic', color='dimgray')

# Save as high-quality PNG
plt.savefig('cias_analysis_high_quality.png', dpi=300, bbox_inches='tight', 
            facecolor='white', edgecolor='none')
plt.savefig('cias_analysis_presentation.png', dpi=300, bbox_inches='tight', 
            facecolor='white', edgecolor='none', transparent=False)

print("High-quality PNG files saved:")
print("- cias_analysis_high_quality.png")
print("- cias_analysis_presentation.png")

# Also create a version with transparent background for presentations
plt.savefig('cias_analysis_transparent.png', dpi=300, bbox_inches='tight', 
            facecolor='none', edgecolor='none', transparent=True)
print("- cias_analysis_transparent.png (transparent background)")

plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D
import matplotlib.patheffects as pe  # for black outline on lines

# ---------------------------------------------------------------------------
# Global plotting style
# ---------------------------------------------------------------------------
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.rcParams["axes.labelsize"] = 16
plt.rcParams["axes.titlesize"] = 18
plt.rcParams["figure.dpi"] = 300

# ---------------------------------------------------------------------------
# Load data
# ---------------------------------------------------------------------------
df_a_workload = pd.read_csv(
    "./results/SETUP_A_EXACT_summary_per_workload.csv"
)
df_b_workload = pd.read_csv(
    "./results/SETUP_B_EXACT_summary_per_workload.csv"
)

# Filter for 5-fold cross-validation only
df_a_workload_5fold = df_a_workload[df_a_workload["n_splits"] == 5]
df_b_workload_5fold = df_b_workload[df_b_workload["n_splits"] == 5]

# Determine window sizes from data (works for 250..1000 or 50..1000)
window_sizes = sorted(
    set(df_a_workload_5fold["window_size"].unique())
    .union(set(df_b_workload_5fold["window_size"].unique()))
)
x_pos = np.arange(len(window_sizes))

# Slightly lighter but still dark colors for each setup
setup_a_color = "#1E88E5"  # Medium-dark blue
setup_b_color = "#8B0000"  # Dark red

# ============================================================================
# FIGURE: MCC, Balanced Accuracy, and Brier vs Window Length
# ============================================================================
metrics = [
    ("mcc", "MCC", 0.5, 1.0),
    ("bal_acc", "Balanced Accuracy", 0.5, 1.0),
    ("brier", "Brier Score", None, None),
]

fig, axes = plt.subplots(
    nrows=len(metrics), ncols=1, figsize=(12, 12), sharex=True
)

for ax, (metric_col, metric_label, y_min, y_max) in zip(axes, metrics):
    # -----------------------------------------------------------------------
    # Scatter points (per workload)
    # -----------------------------------------------------------------------
    for idx, ws in enumerate(window_sizes):
        ws_data_a = df_a_workload_5fold[df_a_workload_5fold["window_size"] == ws]
        ws_data_b = df_b_workload_5fold[df_b_workload_5fold["window_size"] == ws]

        # Setup A scatter
        if not ws_data_a.empty and metric_col in ws_data_a.columns:
            jitter = np.random.uniform(-0.05, 0.05, size=len(ws_data_a))
            ax.scatter(
                [idx + j for j in jitter],
                ws_data_a[metric_col].values,
                facecolor=setup_a_color,
                edgecolor="black",        # thin black border around markers
                alpha=0.7,
                s=55,
                linewidth=0.8,
            )

        # Setup B scatter
        if not ws_data_b.empty and metric_col in ws_data_b.columns:
            jitter = np.random.uniform(-0.05, 0.05, size=len(ws_data_b))
            ax.scatter(
                [idx + j for j in jitter],
                ws_data_b[metric_col].values,
                facecolor=setup_b_color,
                edgecolor="black",        # thin black border around markers
                alpha=0.7,
                s=55,
                linewidth=0.8,
            )

    # -----------------------------------------------------------------------
    # Window-averaged lines (connected across W) with black outline
    # -----------------------------------------------------------------------
    if metric_col in df_a_workload_5fold.columns:
        avg_A = (
            df_a_workload_5fold.groupby("window_size")[metric_col]
            .mean()
            .reindex(window_sizes)
        )
        ax.plot(
            x_pos,
            avg_A.values,
            color=setup_a_color,
            linewidth=2.5,
            marker="o",
            markersize=7,
            markerfacecolor=setup_a_color,
            markeredgecolor="black",
            markeredgewidth=0.8,
            label="Setup A (Window Average)",
            path_effects=[
                pe.Stroke(linewidth=3.5, foreground="black"),
                pe.Normal(),
            ],
        )

    if metric_col in df_b_workload_5fold.columns:
        avg_B = (
            df_b_workload_5fold.groupby("window_size")[metric_col]
            .mean()
            .reindex(window_sizes)
        )
        ax.plot(
            x_pos,
            avg_B.values,
            color=setup_b_color,
            linewidth=2.5,
            linestyle="--",
            marker="s",
            markersize=7,
            markerfacecolor=setup_b_color,
            markeredgecolor="black",
            markeredgewidth=0.8,
            label="Setup B (Window Average)",
            path_effects=[
                pe.Stroke(linewidth=3.5, foreground="black"),
                pe.Normal(),
            ],
        )

    # Axis labels and limits
    ax.set_ylabel(metric_label, fontsize=16, fontweight="bold")
    ax.grid(True, alpha=0.3, color="#AAAAAA", linestyle="-", linewidth=1.0)

    if y_min is not None and y_max is not None:
        ax.set_ylim(y_min, y_max)
        yticks = np.linspace(y_min, y_max, 6)
        ax.set_yticks(yticks)
        ax.set_yticklabels([f"{y:.1f}" for y in yticks])
    else:
        # For Brier, use data-driven limits with a bit of margin
        all_vals = []
        if metric_col in df_a_workload_5fold.columns:
            all_vals.extend(df_a_workload_5fold[metric_col].values)
        if metric_col in df_b_workload_5fold.columns:
            all_vals.extend(df_b_workload_5fold[metric_col].values)
        if len(all_vals) > 0:
            vmin = min(all_vals)
            vmax = max(all_vals)
            margin = 0.1 * (vmax - vmin) if vmax > vmin else 0.01
            ax.set_ylim(max(0.0, vmin - margin), vmax + margin)

# Shared X-axis setup
axes[-1].set_xlabel("Window Size ($W$)", fontsize=16, fontweight="bold")
axes[-1].set_xticks(x_pos)
axes[-1].set_xticklabels(window_sizes)

for ax in axes:
    for spine in ax.spines.values():
        spine.set_edgecolor("#444444")
        spine.set_linewidth(2)
    ax.tick_params(axis="both", which="major", labelsize=16)

fig.suptitle(
    "Window-Level Performance vs Window Size ($W$)\n"
    "MCC, Balanced Accuracy, and Brier Score",
    fontsize=20,
    fontweight="bold",
    y=1.02,
)

plt.tight_layout()
plt.savefig(
    "window_metrics_plot_no_legend.png",
    dpi=300,
    bbox_inches="tight",
    facecolor="white",
)
plt.show()

# ============================================================================
# LEGEND ONLY (two horizontal lines, two entries each)
# ============================================================================
fig2 = plt.figure(figsize=(10, 3))   # medium width, a bit taller for 2 rows
ax2 = fig2.add_subplot(111)

legend_elements = [
    Line2D(
        [0], [0],
        marker="o", color="w", markerfacecolor=setup_a_color,
        markeredgecolor="black", markersize=10,
        label="Setup A (Workloads)", markeredgewidth=0.8,
    ),
    Line2D(
        [0], [0],
        color=setup_a_color, linewidth=2.5, marker="o", markersize=7,
        markerfacecolor=setup_a_color, markeredgecolor="black",
        markeredgewidth=0.8,
        label="Setup A (Window Average)",
    ),
    Line2D(
        [0], [0],
        marker="o", color="w", markerfacecolor=setup_b_color,
        markeredgecolor="black", markersize=10,
        label="Setup B (Workloads)", markeredgewidth=0.8,
    ),
    Line2D(
        [0], [0],
        color=setup_b_color, linewidth=2.5, linestyle="--",
        marker="s", markersize=7,
        markerfacecolor=setup_b_color, markeredgecolor="black",
        markeredgewidth=0.8,
        label="Setup B (Window Average)",
    ),
]

ax2.legend(
    handles=legend_elements,
    loc="center",
    ncol=2,                 # 2 columns → 2 items per row
    fontsize=16,
    framealpha=0.95,
    edgecolor="black",
    frameon=True,
    handlelength=3,
    handletextpad=1.2,
    columnspacing=2.0,
)
ax2.axis("off")

plt.tight_layout()
plt.savefig(
    "window_metrics_legend.png",
    dpi=300,
    bbox_inches="tight",
    facecolor="white",
)
plt.show()